## Package

In [38]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from huggingface_hub import login
import json
import os
import time
from datetime import datetime
from tqdm import tqdm
import re

## Get Data

In [39]:
# 資料讀取選項
# 您可以選擇以下任一種方式來載入資料：

# 選項 1: 從已儲存的本地檔案讀取 (推薦，速度快)
use_local_files = False

# 選項 2: 從 Hugging Face 直接串流載入 (需要網路連線)
use_streaming = True

# 選項 3: 下載完整資料集 (檔案很大，不推薦)
use_full_download = False

print("=== 資料載入選項 ===")
print(f"使用本地檔案: {use_local_files}")
print(f"使用串流模式: {use_streaming}")
print(f"下載完整資料集: {use_full_download}")

# 資料載入
if use_local_files:
    print("\n📁 從本地檔案讀取資料...")
    
    # 檢查已儲存的檔案
    save_dir = "saved_datasets"
    
    if os.path.exists(save_dir):
        import glob
        
        # 尋找可用的檔案
        csv_files = glob.glob(f"{save_dir}/*.csv")
        json_files = glob.glob(f"{save_dir}/*.json")
        parquet_files = glob.glob(f"{save_dir}/*.parquet")
        
        print(f"找到的檔案:")
        print(f"  CSV 檔案: {len(csv_files)} 個")
        print(f"  JSON 檔案: {len(json_files)} 個")
        print(f"  Parquet 檔案: {len(parquet_files)} 個")
        
        # 優先使用 Parquet 檔案 (最高效)
        if parquet_files:
            latest_file = max(parquet_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 Parquet 檔案: {latest_file}")
            df = pd.read_parquet(latest_file)
            
        # 其次使用 CSV 檔案
        elif csv_files:
            latest_file = max(csv_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 CSV 檔案: {latest_file}")
            df = pd.read_csv(latest_file)
            
        # 最後使用 JSON 檔案
        elif json_files:
            latest_file = max(json_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 JSON 檔案: {latest_file}")
            with open(latest_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            df = pd.DataFrame(data)
            
        else:
            print("❌ 沒有找到已儲存的資料檔案")
            print("請先執行資料下載和儲存的程式碼")
            df = None
    else:
        print("❌ 找不到 saved_datasets 目錄")
        print("請先執行資料下載和儲存的程式碼")
        df = None

elif use_streaming:
    print("\n🌐 從 Hugging Face 串流載入資料...")
    
    # 使用串流模式載入資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset", streaming=True)
    
    # 設定要載入的樣本數量 - 減少到100筆用於演示
    num_samples = 1000
    print(f"載入前 {num_samples} 筆資料...")
    
    # 收集資料
    sample_data = []
    for i, example in enumerate(dataset['train']):
        if i >= num_samples:
            break
        sample_data.append(example)
        if (i + 1) % 25 == 0:
            print(f"  已載入 {i + 1} 筆資料...")
    
    # 轉換為 DataFrame
    df = pd.DataFrame(sample_data)
    
elif use_full_download:
    print("\n⬇️ 下載完整資料集...")
    print("警告：這將下載 13.7GB 的資料，可能需要很長時間")
    
    # 下載完整資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset")
    df = dataset['train'].to_pandas()

else:
    print("❌ 沒有選擇任何資料載入選項")
    df = None

# 顯示資料資訊
if df is not None:
    print(f"\n✅ 資料載入成功！")
    print(f"📊 資料形狀: {df.shape}")
    print(f"📋 欄位名稱: {list(df.columns)}")
    
    # 顯示基本統計
    if 'text' in df.columns: # type: ignore
        df['text_length'] = df['text'].str.len() # type: ignore
        print(f"\n📈 文本長度統計:")
        print(df['text_length'].describe()) # type: ignore
        
        # 顯示前幾筆資料範例
        print(f"\n📝 前 3 筆資料範例:")
        for i in range(min(3, len(df))): # type: ignore
            text = df.iloc[i]['text']
            # 顯示前100個字符
            preview = text[:100] + "..." if len(text) > 100 else text
            print(f"範例 {i+1} ({len(text)} 字符): {preview}")
            print("-" * 80)
    
    print(f"\n🎯 資料已準備就緒，可用於後續的 LLM 評分處理！")
else:
    print("\n❌ 資料載入失敗，請檢查設定並重新執行")

# 儲存到全域變數供後續使用
globals()['dataset_df'] = df

=== 資料載入選項 ===
使用本地檔案: False
使用串流模式: True
下載完整資料集: False

🌐 從 Hugging Face 串流載入資料...
載入前 1000 筆資料...
  已載入 25 筆資料...
  已載入 50 筆資料...
  已載入 75 筆資料...
  已載入 100 筆資料...
  已載入 125 筆資料...
  已載入 150 筆資料...
  已載入 175 筆資料...
  已載入 200 筆資料...
  已載入 225 筆資料...
  已載入 250 筆資料...
  已載入 275 筆資料...
  已載入 300 筆資料...
  已載入 325 筆資料...
  已載入 350 筆資料...
  已載入 375 筆資料...
  已載入 400 筆資料...
  已載入 425 筆資料...
  已載入 450 筆資料...
  已載入 475 筆資料...
  已載入 500 筆資料...
  已載入 525 筆資料...
  已載入 550 筆資料...
  已載入 575 筆資料...
  已載入 600 筆資料...
  已載入 625 筆資料...
  已載入 650 筆資料...
  已載入 675 筆資料...
  已載入 700 筆資料...
  已載入 725 筆資料...
  已載入 750 筆資料...
  已載入 775 筆資料...
  已載入 800 筆資料...
  已載入 825 筆資料...
  已載入 850 筆資料...
  已載入 875 筆資料...
  已載入 900 筆資料...
  已載入 925 筆資料...
  已載入 950 筆資料...
  已載入 975 筆資料...
  已載入 1000 筆資料...

✅ 資料載入成功！
📊 資料形狀: (1000, 1)
📋 欄位名稱: ['text']

📈 文本長度統計:
count     1000.000000
mean       300.899000
std        785.763282
min          5.000000
25%         38.000000
50%        105.500000
75%        274.000000
max      1

In [40]:

# save_dir = "saved_datasets"
# os.makedirs(save_dir, exist_ok=True)
# df.to_csv(os.path.join(save_dir, "cluecorpus.csv"), index=False, encoding="utf-8-sig")


## 📝 文本切分處理

In [41]:
# 📝 文本切分處理 - 切分為句子級別的片段
print("🔪 啟動文本切分處理...")

def split_text_to_sentences(text, min_length=30, max_length=250):
    """
    將文本切分為句子級別的片段
    
    Args:
        text (str): 原始文本
        min_length (int): 最小片段長度
        max_length (int): 最大片段長度
    
    Returns:
        list: 切分後的句子片段列表
    """
    # 定義標點符號分隔符
    sentence_separators = ['。', '！', '？', '；', '…']  # 強句號分隔符
    phrase_separators = ['，', '、', '：', '；']  # 弱分隔符
    
    # 1. 首先按強標點符號切分成句子
    sentences = []
    current_sentence = ""
    
    for char in text:
        current_sentence += char
        if char in sentence_separators:
            if current_sentence.strip():
                sentences.append(current_sentence.strip())
                current_sentence = ""
    
    # 處理最後一個句子（如果沒有以強標點結尾）
    if current_sentence.strip():
        sentences.append(current_sentence.strip())
    
    # 2. 對每個句子進一步按逗號等分隔符切分
    fragments = []
    
    for sentence in sentences:
        # 跳過太短的句子
        if len(sentence) < min_length:
            continue
            
        # 如果句子長度在合理範圍內，直接使用
        if len(sentence) <= max_length:
            fragments.append(sentence)
        else:
            # 對長句子按逗號等進一步切分
            parts = []
            current_part = ""
            
            for char in sentence:
                current_part += char
                if char in phrase_separators:
                    if current_part.strip() and len(current_part.strip()) >= min_length:
                        parts.append(current_part.strip())
                        current_part = ""
            
            # 處理最後一部分
            if current_part.strip() and len(current_part.strip()) >= min_length:
                parts.append(current_part.strip())
            
            # 如果切分後的部分太短，嘗試合併
            merged_parts = []
            temp_part = ""
            
            for part in parts:
                if len(temp_part + part) <= max_length:
                    temp_part = temp_part + part if temp_part else part
                else:
                    if temp_part and len(temp_part) >= min_length:
                        merged_parts.append(temp_part)
                    temp_part = part
            
            # 添加最後一部分
            if temp_part and len(temp_part) >= min_length:
                merged_parts.append(temp_part)
            
            # 如果切分失敗，直接截斷
            if not merged_parts and len(sentence) >= min_length:
                # 簡單截斷成合適長度的片段
                for i in range(0, len(sentence), max_length):
                    fragment = sentence[i:i+max_length]
                    if len(fragment) >= min_length:
                        merged_parts.append(fragment)
            
            fragments.extend(merged_parts)
    
    return fragments

def split_text_by_punctuation(text, min_length=30, max_length=250):
    """
    兼容性函數 - 調用新的句子切分函數
    """
    return split_text_to_sentences(text, min_length, max_length)

def process_text_splitting(df, text_column='text', min_length=30, max_length=250):
    """
    處理整個資料集的文本切分 - 句子級別切分
    
    Args:
        df (DataFrame): 原始資料集
        text_column (str): 文本欄位名稱
        min_length (int): 最小句子片段長度
        max_length (int): 最大句子片段長度
    
    Returns:
        DataFrame: 切分後的資料集
    """
    print(f"📊 開始處理文本切分...")
    print(f"  原始資料筆數: {len(df)}")
    print(f"  文本欄位: {text_column}")
    print(f"  句子片段長度範圍: {min_length}-{max_length} 字")
    
    split_data = []
    total_fragments = 0
    processed_texts = 0
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="切分進度"):
        original_text = row[text_column]
        
        # 跳過太短的文本
        if len(original_text) < min_length:
            continue
        
        search_start_pos = 0
        # 切分文本為句子片段
        fragments = split_text_to_sentences(original_text, min_length, max_length)
        
        # 為每個片段創建新記錄
        for frag_idx, fragment in enumerate(fragments):
            # 嘗試在原文中找到該片段的位置（從 search_start_pos 往後找）
            start_pos = original_text.find(fragment, search_start_pos)
            end_pos = start_pos + len(fragment) if start_pos != -1 else -1
            # 更新下一次搜尋起點（防止重複片段誤配）
            if start_pos != -1:
                search_start_pos = end_pos


            new_row = row.copy()
            new_row[text_column] = fragment
            new_row['original_index'] = idx
            new_row['fragment_index'] = frag_idx
            new_row['original_text_length'] = len(original_text)
            new_row['fragment_length'] = len(fragment)
            new_row['source_type'] = 'sentence_fragment'
            #新增
            new_row['fragment_start'] = start_pos
            new_row['fragment_end'] = end_pos

            split_data.append(new_row)
            total_fragments += 1
        
        processed_texts += 1
    
    # 創建新的DataFrame
    split_df = pd.DataFrame(split_data)
    
    print(f"\n✅ 文本切分完成！")
    print(f"📈 切分統計:")
    print(f"  處理文本數: {processed_texts}")
    print(f"  生成句子片段數: {total_fragments}")
    print(f"  平均每文本片段數: {total_fragments/processed_texts:.1f}")
    
    if not split_df.empty:
        print(f"\n📏 片段長度統計:")
        length_stats = split_df['fragment_length'].describe()
        print(f"  平均長度: {length_stats['mean']:.1f} 字")
        print(f"  最短片段: {length_stats['min']:.0f} 字")
        print(f"  最長片段: {length_stats['max']:.0f} 字")
        print(f"  中位數長度: {length_stats['50%']:.1f} 字")
        
        # 長度分布
        length_ranges = [
            (10, 20, "短片段"),
            (20, 30, "中短片段"),
            (30, 40, "中等片段"),
            (40, 50, "長片段"),
            (50, 100, "超長片段")
        ]
        
        print(f"\n📊 片段長度分布:")
        for min_len, max_len, desc in length_ranges:
            count = len(split_df[(split_df['fragment_length'] >= min_len) & 
                                (split_df['fragment_length'] < max_len)])
            percentage = count / len(split_df) * 100
            print(f"  {desc} ({min_len}-{max_len}字): {count} 個 ({percentage:.1f}%)")
    
    return split_df

# 執行文本切分
if 'dataset_df' in globals() and dataset_df is not None:
    print(f"🎯 對載入的資料集進行句子級別文本切分...")
    
    # 設定切分參數 - 改為句子級別
    MIN_FRAGMENT_LENGTH = 30   # 最小片段長度
    MAX_FRAGMENT_LENGTH = 250   # 最大片段長度
    
    print(f"\n⚙️ 切分參數:")
    print(f"  最小片段長度: {MIN_FRAGMENT_LENGTH} 字")
    print(f"  最大片段長度: {MAX_FRAGMENT_LENGTH} 字")
    print(f"  切分模式: 句子級別")
    
    # 執行切分
    split_dataset_df = process_text_splitting(
        df=dataset_df, 
        text_column='text',
        min_length=MIN_FRAGMENT_LENGTH,
        max_length=MAX_FRAGMENT_LENGTH
    )
    
    if not split_dataset_df.empty:
        # 顯示切分範例
        print(f"\n📝 切分範例:")
        print("="*80)
        
        # 找一個有多個片段的原始文本
        sample_original_idx = split_dataset_df['original_index'].value_counts().index[0]
        sample_fragments = split_dataset_df[split_dataset_df['original_index'] == sample_original_idx]
        
        print(f"原始文本 #{sample_original_idx} 被切分為 {len(sample_fragments)} 個句子片段:")
        print()
        
        # 顯示原始文本
        original_text = dataset_df.iloc[sample_original_idx]['text']
        print(f"原始文本: {original_text[:200]}{'...' if len(original_text) > 200 else ''}")
        print()
        print("切分結果:")
        
        for i, (_, row) in enumerate(sample_fragments.iterrows()):
            fragment = row['text']
            length = row['fragment_length']
            print(f"片段 {i+1} ({length}字): {fragment}")
            print("-" * 60)
        
        # 儲存切分後的資料集
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        split_dir = "split_datasets"
        os.makedirs(split_dir, exist_ok=True)
        
        # 儲存為多種格式
        base_filename = f"{split_dir}/sentence_fragments_{timestamp}"
        
        # CSV 格式
        csv_filename = f"{base_filename}.csv"
        split_dataset_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        
        # JSON 格式
        json_filename = f"{base_filename}.json"
        split_dataset_df.to_json(json_filename, orient='records', force_ascii=False, indent=2)
        
        # Parquet 格式
        parquet_filename = f"{base_filename}.parquet"
        split_dataset_df.to_parquet(parquet_filename, index=False)
        
        print(f"\n💾 句子片段資料集已儲存:")
        print(f"  📄 CSV: {csv_filename}")
        print(f"  📋 JSON: {json_filename}")
        print(f"  📦 Parquet: {parquet_filename}")
        
        # 檔案大小統計
        print(f"\n📁 檔案大小:")
        for filename in [csv_filename, json_filename, parquet_filename]:
            if os.path.exists(filename):
                size_mb = os.path.getsize(filename) / (1024 * 1024)
                print(f"  {os.path.basename(filename)}: {size_mb:.2f} MB")
        
        # 儲存到全域變數
        globals()['split_dataset_df'] = split_dataset_df
        
        print(f"\n🎉 句子級別文本切分處理完成！")
        print(f"📋 變數名稱: split_dataset_df")
        print(f"🎯 句子片段資料集可用於後續的 LLM 處理！")
    
    else:
        print("❌ 切分後沒有產生有效片段，請檢查原始資料")
        split_dataset_df = None

else:
    print("❌ 沒有找到資料集，請先執行 Get Data 部分")
    split_dataset_df = None

print("="*80)

🔪 啟動文本切分處理...
🎯 對載入的資料集進行句子級別文本切分...

⚙️ 切分參數:
  最小片段長度: 30 字
  最大片段長度: 250 字
  切分模式: 句子級別
📊 開始處理文本切分...
  原始資料筆數: 1000
  文本欄位: text
  句子片段長度範圍: 30-250 字


切分進度: 100%|██████████| 1000/1000 [00:02<00:00, 403.54it/s]



✅ 文本切分完成！
📈 切分統計:
  處理文本數: 794
  生成句子片段數: 3870
  平均每文本片段數: 4.9

📏 片段長度統計:
  平均長度: 62.9 字
  最短片段: 30 字
  最長片段: 511 字
  中位數長度: 52.0 字

📊 片段長度分布:
  短片段 (10-20字): 0 個 (0.0%)
  中短片段 (20-30字): 0 個 (0.0%)
  中等片段 (30-40字): 1016 個 (26.3%)
  長片段 (40-50字): 744 個 (19.2%)
  超長片段 (50-100字): 1673 個 (43.2%)

📝 切分範例:
原始文本 #51 被切分為 208 個句子片段:

原始文本: 发改委力挺节能环保业 节能环保产业10大概念股价值解析导读：发改委力挺节能环保业"绿色经济"元年到来节能环保10万亿环保蛋糕谁来分节能环保产业行动计划今年将出力挺环保产业发展部委将严查数据造假环保业或迎黄金时代环保税开征渐行渐近国元证券：三主线掘金7只节能环保股节能环保产业10大概念股价值解析发改委力挺节能环保业 "绿色经济"元年到来从国家发改委获悉，上周五(4月10日)，国家发改委召开全国发展改...

切分結果:
片段 1 (248字): 发改委力挺节能环保业 节能环保产业10大概念股价值解析导读：发改委力挺节能环保业"绿色经济"元年到来节能环保10万亿环保蛋糕谁来分节能环保产业行动计划今年将出力挺环保产业发展部委将严查数据造假环保业或迎黄金时代环保税开征渐行渐近国元证券：三主线掘金7只节能环保股节能环保产业10大概念股价值解析发改委力挺节能环保业 "绿色经济"元年到来从国家发改委获悉，上周五(4月10日)，国家发改委召开全国发展改革系统资源节约和环境保护工作电视电话会议，国家发展改革委党组成员、副主任张勇同志出席会议并讲话。
------------------------------------------------------------
片段 2 (64字): 张勇指出，2015年的环资工作要全面落实党中央、国务院的决策部署，按照全国发展和改革工作会议的安排，明确目标任务，扎扎实实推进。
------------------------------------------------------

In [42]:
split_dataset_df.head()

,text,text_length,original_index,fragment_index,original_text_length,fragment_length,source_type,fragment_start,fragment_end
3,兰州公交集团:明起至8月底三条公交线暂不经过靖远路站原标题：明起至8月底三条公交线暂不经过靖...,545,3,0,545,155,sentence_fragment,0,155
3,据悉，35路公交线由兰州西客站开往美伦广场时，兰州西客站至白塔山公园线路不变，经北滨河路九州...,545,3,1,545,71,sentence_fragment,155,226
3,由美伦广场开往兰州西客站时，美伦广场至庙滩子线路不变，左转弯进入九州大道至北滨河路市二医院恢...,545,3,2,545,53,sentence_fragment,226,279
3,81路公交线路由市二医院开往省军干所时，经北滨河路九州大道南口左转弯进入九州大道至庙滩子恢复...,545,3,3,545,52,sentence_fragment,279,331
3,由省军干所开往市二医院时，省军干所至庙滩子线路不变，左转弯进入九州大道至市二医院。,545,3,4,545,41,sentence_fragment,331,372


## LLM AUG

In [43]:
API_KEY ="sk-b56c488f33b94df297a6314bd037b805"

## QWEN-100
MODEL_NAME = "Qwen3-30B-A3B"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

In [10]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio



# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"],"酸奶": ["優格"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "牛逼": ["超強"], "給力": ["很棒"], "微信": [""], "支付寶": [""], "淘寶": [""],
    "視頻" :["影片"],"屏幕":["螢幕"]	
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }


async def mainland_score_api_async(text, session, model_endpoint, api_key, model_name):
    """使用你提供的 API 非同步推論大陸用語分數"""

    prompt = f"""評估文本的大陸用語特徵，每項0或1分：

文本：{text}

評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 100
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            if response.status != 200:
                return f"[ERROR] API HTTP 狀態碼: {response.status}"
            
            data = await response.json()
            
            # 加這個，看看整包回傳長怎樣
            return f"[DEBUG RAW RESPONSE]\n{json.dumps(data, indent=2, ensure_ascii=False)}"

    except Exception as e:
        return f"[EXCEPTION] {str(e)}"


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="Qwen3-30B-A3B",
                                        text_col='text', sample_size=100, threshold=3, batch_size=20):
    
    
    print(f"📊 處理資料集：{len(df)} 筆")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    indices = sample_df.index.tolist()

    results = []
    authentic_texts = []
    generic_texts = []

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)
            for i, response in enumerate(responses):
                print(f"DEBUG response {i}:\n{response}\n{'-'*40}")

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length']
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "Qwen3-30B-A3B"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 3870 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：3870 筆


非同步批次推論:  20%|██        | 1/5 [00:01<00:07,  1.83s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-05bc030c5b714dd5bab090e650beece0",
  "created": 1757379078,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本中的大陆用语特征，并按照给定的五个标准进行评分，每项0或1分。首先，我需要仔细阅读文本，然后逐一对照每个评分标准。\n\n首先看文本内容：“现在已经成了吃小海鲜就会想到的地方因为去旁边的锅一烧而路过的地方，本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了一跳，还真是便宜的说现在已经吃遍了这里"
      },
      "provider_specific_fields": {}
    }
  ],
  "usage": {
    "completion_tokens": 100,
    "prompt_tokens": 252,
    "total_tokens": 352,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "service_tier": null,
  "prompt_logprobs": null,
  "kv_transfer_params": null
}
----------------------------------------
DEBUG response 1:


非同步批次推論:  40%|████      | 2/5 [00:02<00:04,  1.43s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-772007f199cf4f19a5a5abeacf8b81b2",
  "created": 1757379082,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本中的大陆用语特征，并按照给定的五个标准进行评分。首先，我需要仔细阅读文本，然后逐一检查每个标准。\n\n文本内容是：“一开始不知道这里是吃什么的，很少来这里消费，现拿手机上点评查的，哈哈，后来进来之后就觉得真是仅对了诶，没想到这里有拉面炒年糕，哈哈，一直想在北京找一家正宗的拉面炒年糕店，于是点了"
      },
      "provider_specific_fields": {}
    }
  ],
  "usage": {
    "completion_tokens": 100,
    "prompt_tokens": 235,
    "total_tokens": 335,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "service_tier": null,
  "prompt_logprobs": null,
  "kv_transfer_params": null
}
----------------------------------------
DEBUG response 1:
[

...

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-1499d0383fc247cb812e9f17cc9db86a",
  "created": 1757379086,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陸用語特徵，并按照给定的五个标准进行评分，每项0或1分。首先，我要仔细阅读文本，然后逐一检查每个标准。\n\n文本是：“根据飞行中的营养标准，组合体阶段每名航天员每天需要的热量‘折合’成食物的分量，相当于一到两公斤。”\n\n首先看第一个标准：大陸特有詞彙。用户给出的例子包括計算機"
      },
      "provider_specific_fields": {}
    }
  ],
  "usage": {
    "completion_tokens": 100,
    "prompt_tokens": 208,
    "total_tokens": 308,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "service_tier": null,
  "prompt_logprobs": null,
  "kv_transfer_params": null
}
----------------------------------------
DEBUG response 1:
[DEBUG

非同步批次推論:  80%|████████  | 4/5 [00:07<00:01,  1.72s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-f84cb2cb63e74440b9c98dc34bbb9857",
  "created": 1757379090,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本中的大陆用语特征，并按照给定的评分标准进行打分。首先，我需要仔细阅读用户提供的文本，然后逐一检查每个评分标准。\n\n文本内容是：“莆田市在全市机关中开展‘读书、荐书、品书’活动，利用农家书屋在广大乡村开展以‘我的书屋、我的梦’为主题的阅读活动，开设‘夫妻课堂’‘婆媳课堂’‘亲子课堂’"
      },
      "provider_specific_fields": {}
    }
  ],
  "usage": {
    "completion_tokens": 100,
    "prompt_tokens": 236,
    "total_tokens": 336,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "service_tier": null,
  "prompt_logprobs": null,
  "kv_transfer_params": null
}
----------------------------------------
DEBUG response 1:
[DEBUG

非同步批次推論: 100%|██████████| 5/5 [00:08<00:00,  1.65s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-011fd3b8b18a45f9924ea48c628cc92c",
  "created": 1757379094,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陆用语特征，按照给定的五个评分标准，每项0或1分。首先，我需要仔细阅读文本，然后逐一分析每个标准。\n\n文本内容是：“和老公两个人怎么吃也超不过100大米由于经常去，和几个服务员都混的比较熟了，其实他们的服务态度还真是挺好的，就是服务员的数量太少啦所以有时候忙不过来。”\n\n首先看第一个标准：大陆"
      },
      "provider_specific_fields": {}
    }
  ],
  "usage": {
    "completion_tokens": 100,
    "prompt_tokens": 220,
    "total_tokens": 320,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "service_tier": null,
  "prompt_logprobs": null,
  "kv_transfer_params": null
}
----------------------------------------
DEBUG response

## QWEN-512
MODEL_NAME = "Qwen3-30B-A3B"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

In [11]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio



# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"],"酸奶": ["優格"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "牛逼": ["超強"], "給力": ["很棒"], "微信": [""], "支付寶": [""], "淘寶": [""],
    "視頻" :["影片"],"屏幕":["螢幕"]	
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }


async def mainland_score_api_async(text, session, model_endpoint, api_key, model_name):
    """使用你提供的 API 非同步推論大陸用語分數"""

    prompt = f"""評估文本的大陸用語特徵，每項0或1分：

文本：{text}

評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 512
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            if response.status != 200:
                return f"[ERROR] API HTTP 狀態碼: {response.status}"
            
            data = await response.json()
            
            # 加這個，看看整包回傳長怎樣
            return f"[DEBUG RAW RESPONSE]\n{json.dumps(data, indent=2, ensure_ascii=False)}"

    except Exception as e:
        return f"[EXCEPTION] {str(e)}"


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="Qwen3-30B-A3B",
                                        text_col='text', sample_size=100, threshold=3, batch_size=20):
    
    
    print(f"📊 處理資料集：{len(df)} 筆")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    indices = sample_df.index.tolist()

    results = []
    authentic_texts = []
    generic_texts = []

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)
            for i, response in enumerate(responses):
                print(f"DEBUG response {i}:\n{response}\n{'-'*40}")

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length']
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "Qwen3-30B-A3B"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 3870 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：3870 筆


非同步批次推論:  20%|██        | 1/5 [00:17<01:11, 17.79s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-8c2b4516728c47d8bb6808b1b7ff2da5",
  "created": 1757379307,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本中的大陆用语特征，并按照给定的评分标准进行打分。首先，我需要仔细阅读文本，然后逐一检查每个评分标准。\n\n文本内容是：“现在已经成了吃小海鲜就会想到的地方因为去旁边的锅一烧而路过的地方，本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了一跳，还真是便宜的说现在已经吃遍了这里所有的菜和主食，写下自己不喜欢的，剩下的都OK：卤水蚍蛴香螺因为我比较喜欢吃酱爆口味的。”\n\n首先看第一个标准：大陆特有词汇，比如计算机、软件、出租车、地铁等。文本中提到的“小海鲜”、“锅一烧”、“倪式海泰”、“菜单”、“卤水蚍蛴香螺”这些词，看起来像是食物或餐厅相关的词汇，但“倪式海泰”可能是一个餐厅的名字，而“卤水蚍蛴香螺”可能是一种海鲜菜品。不过这些词汇是否属于大陆特有的呢？“小海鲜”可能在大陆比较常见，但不确定是否属于特有词汇。其他如“菜单”、“吃遍”可能比较通用，没有明显的大陆特有词汇。所以可能这一项得0分。\n\n接下来是大陆语法习惯，比如“挺...的”、“蛮...的”、“咋样”等。文本中有没有这样的结构？比如“本以为是...会很贵，谁知道一进去就被菜单吓了一跳”这里用了“本以为...谁知道...”的结构，可能属于常见的表达方式，但不确

非同步批次推論:  40%|████      | 2/5 [00:33<00:50, 16.83s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-f9cb99f7bd54424e841f917b44437981",
  "created": 1757379324,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\n大陸特有詞彙:0  \n大陸語法習慣:0  \n大陸口語表達:0  \n避免繁體用語:1  \n整體大陸化程度:0  \n總分:1",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本中的大陆用语特征，并按照给定的评分标准进行打分。首先，我需要仔细阅读文本，然后逐一检查每个评分标准。\n\n文本内容是：“一开始不知道这里是吃什么的，很少来这里消费，现拿手机上点评查的，哈哈，后来进来之后就觉得真是仅对了诶，没想到这里有拉面炒年糕，哈哈，一直想在北京找一家正宗的拉面炒年糕店，于是点了试吃。”\n\n首先看第一个标准：大陆特有词汇。需要检查是否有计算机、软件、出租车、地铁等词汇。文本中提到的“手机”、“点评”可能是指大众点评，但“点评”本身可能不算特有词汇，而“手机”在大陆和台湾都通用。没有出现标准中列出的特有词汇，所以这里应该是0分。\n\n接下来是大陆语法习惯，比如“挺...的”、“蛮...的”、“咋样”等。文本中没有这些结构，比如“挺好的”或“蛮喜欢的”，所以这里也是0分。\n\n第三个是大陆口语表达，如“搞定”、“整”、“弄”等。文本中没有这些词，所以也是0分。\n\n第四个是避免繁体用语，检查是否有“电脑”、“软件”、“资料”等。文本中没有这些词，所以这里给1分。\n\n最后

...

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-e1ca079e11a9400eb7493a8a9d5f1a70",
  "created": 1757379340,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": "\n\n大陸特有詞彙:0  \n大陸語法習慣:0  \n大陸口語表達:0",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陸用語特徵，并按照给定的五个标准进行评分，每项0或1分。首先，我要仔细阅读用户提供的文本和评分标准，确保自己正确理解每个评分项的含义。\n\n首先看文本：“根据飞行中的营养标准，组合体阶段每名航天员每天需要的热量‘折合’成食物的分量，相当于一到两公斤。” 这句话看起来比较正式，可能来自科技或官方文件。\n\n接下来是评分标准：\n\n1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等。需要检查文本中是否有这些词汇。例如，“計算機”是繁体，而大陆用“计算机”，但这里文本中没有出现这些词，所以可能得0分。\n\n2. 大陸語法習慣：挺...的、蠻...的、咋樣等。检查是否有这些结构。文本中没有这些表达，所以可能0分。\n\n3. 大陸口語表達：搞定、整、弄等。文本中也没有这些口语词，所以0分。\n\n4. 避免繁體用語：不含電腦、軟體、資料等。需要确认文本中是否有这些繁体字。例如，“電腦”是繁体，而大陆用“计算机”；“軟體”对应“软件”，“資料”对应“数据”或“资料”。文本中是否有这些？原句中的“折合”是正确的简体字，没有出现“電腦”、“軟體”或“資料”，所以可

非同步批次推論:  80%|████████  | 4/5 [01:06<00:16, 16.39s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-40e63e22b7574416a5d9bf5a3399891a",
  "created": 1757379356,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\n大陸特有詞彙:0  \n大陸語法習慣:0  \n大陸口語表達:0  \n避免繁體用語:1  \n整體大陸化程度:0  \n總分:1",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本中的大陆用语特征，并按照给定的评分标准进行打分。首先，我要仔细阅读用户提供的文本，然后逐一检查每个评分标准。\n\n文本内容是：“莆田市在全市机关中开展‘读书、荐书、品书’活动，利用农家书屋在广大乡村开展以‘我的书屋、我的梦’为主题的阅读活动，开设‘夫妻课堂’‘婆媳课堂’‘亲子课堂’‘祖辈家长课堂’，等等。”\n\n首先看第一个标准：大陆特有词汇。用户给出的例子包括计算机、软件、出租车、地铁等。我需要检查文本中是否有这些词汇。文本中提到的“农家书屋”可能是一个大陆特有的概念，但不确定是否在标准列表中。其他词汇如“读书、荐书、品书”可能比较常见，但可能不算特有词汇。而“夫妻课堂”“婆媳课堂”等可能属于活动名称，但不确定是否属于特有词汇。根据用户的标准，可能没有直接出现例子中的词汇，所以可能得0分。\n\n接下来是大陆语法习惯，比如“挺...的”“蛮...的”“咋样”等。文本中没有这些结构，所以语法习惯可能也是0分。\n\n第三项是大陆口语表达，如“搞定、整、弄”等。文本中没有这些词汇，所以口语表达也是

非同步批次推論: 100%|██████████| 5/5 [01:23<00:00, 16.62s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-46992b6af09d41a6ac75e15fb20e544d",
  "created": 1757379373,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陆用语特征，按照给定的五个评分标准，每项0或1分。首先，我需要仔细阅读文本，然后逐一检查每个标准。\n\n文本内容是：“和老公两个人怎么吃也超不过100大米由于经常去，和几个服务员都混的比较熟了，其实他们的服务态度还真是挺好的，就是服务员的数量太少啦所以有时候忙不过来。”\n\n首先看第一个标准：大陆特有词汇。用户提供的例子包括计算机、软件、出租车、地铁等。我需要检查文本中是否有这些词汇。文本中提到的“大米”可能是指“米饭”，但“大米”本身并不是大陆特有的词汇，而是普遍使用的词汇。其他词汇如“老公”、“服务员”、“服务态度”等，虽然常见，但可能不算大陆特有的。所以这里可能没有符合的词汇，所以第一项应该是0分。\n\n接下来是第二个标准：大陆语法习惯，比如“挺...的”、“蛮...的”、“咋样”等。文本中出现了“还真是挺好的”，这里的“挺”符合“挺...的”的结构，所以第二项应该得1分。\n\n第三个标准是大陆口语表达，如“搞定、整、弄”等。文本中没有出现这些词，所以第三项是0分。\n\n第四个标准是避免繁体用语，即不包含“电脑、软件、资料”等。文本中没有这些繁体字词汇，所以第四项是1分。\n\n第五个是整体大陆化程度，综合

## Qwen-512

MODEL_NAME = "Qwen3-30B-A3B"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

In [13]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio



# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }


async def mainland_score_api_async(text, session, model_endpoint, api_key, model_name):
    """使用你提供的 API 非同步推論大陸用語分數"""

    prompt = f"""
評估文本的大陸用語特徵，每項0或1分：


文本：{text}


評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等 
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估

請回傳格式如下：

大陸特有詞彙:
大陸語法習慣:
大陸口語表達:
避免繁體用語:
整體大陸化程度:
總分:
"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 512
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            if response.status != 200:
                return f"[ERROR] API HTTP 狀態碼: {response.status}"
            
            data = await response.json()
            
            # 加這個，看看整包回傳長怎樣
            return f"[DEBUG RAW RESPONSE]\n{json.dumps(data, indent=2, ensure_ascii=False)}"

    except Exception as e:
        return f"[EXCEPTION] {str(e)}"


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="Qwen3-30B-A3B",
                                        text_col='text', sample_size=100, threshold=3, batch_size=20):
    
    
    print(f"📊 處理資料集：{len(df)} 筆")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    indices = sample_df.index.tolist()

    results = []
    authentic_texts = []
    generic_texts = []

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)
            for i, response in enumerate(responses):
                print(f"DEBUG response {i}:\n{response}\n{'-'*40}")

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                # 找到切割文本
                original_row = available_data.iloc[idx]
                start_position = original_row.get('fragment_start', -1)
                end_position = original_row.get('fragment_end', -1)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'start_position': start_position,  # 新增字串的起始位置
                    'end_position': end_position,  # 新增字串的結束位置
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'index': r['index'],  # 儲存索引
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms']),
            'start_position': r['start_position'],  # 儲存起始位置
            'end_position': r['end_position']  # 儲存結束位置
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length'],
                'start_position': r['start_position'],  # 加入起始位置
                'end_position': r['end_position']  # 加入結束位置
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "Qwen3-30B-A3B"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 3870 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：3870 筆


非同步批次推論:  20%|██        | 1/5 [00:16<01:07, 16.77s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-506d627409834a3bb187ed98603dfd16",
  "created": 1757380239,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本中的大陆用语特征，并按照给定的五个标准进行评分，每项0或1分。首先，我要仔细阅读文本，理解每个部分的内容，然后逐一对照评分标准。\n\n首先看文本内容：“现在已经成了吃小海鲜就会想到的地方因为去旁边的锅一烧而路过的地方，本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了一跳，还真是便宜的说现在已经吃遍了这里所有的菜和主食，写下自己不喜欢的，剩下的都OK：卤水蚍蛴香螺因为我比较喜欢吃酱爆口味的。”\n\n接下来，我需要检查每个评分标准：\n\n1. 大陆特有词汇：需要检查是否有计算机、软件、出租车、地铁等词汇。文本中没有出现这些词，所以可能得0分。\n\n2. 大陆语法习惯：比如“挺...的”、“蛮...的”、“咋样”等。文本中是否有这样的结构？比如“挺贵的”或者“蛮便宜的”？但原文中“本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了一跳，还真是便宜的说”这里用了“便宜的说”，可能接近“的说”这种口语表达，但不确定是否符合标准中的“挺...的、蛮...的、咋样等”。可能没有明显的例子，所以可能得0分。\n\n3. 大陆口语表达：比如“搞定、整、弄等”。文本中是否有这些词？比如“吃遍了这里所有的菜和主食”中的“吃遍”可能

非同步批次推論:  40%|████      | 2/5 [00:32<00:49, 16.35s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-98676beacd974f49b7184b5504fc86b2",
  "created": 1757380255,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本中的大陆用语特征，并按照给定的五个标准进行评分，每项0或1分。首先，我需要仔细阅读文本，然后逐一对照每个评分标准。\n\n首先看文本内容：“一开始不知道这里是吃什么的，很少来这里消费，现拿手机上点评查的，哈哈，后来进来之后就觉得真是仅对了诶，没想到这里有拉面炒年糕，哈哈，一直想在北京找一家正宗的拉面炒年糕店，于是点了试吃。”\n\n接下来，我需要检查每个评分标准：\n\n1. 大陆特有词汇：需要检查是否有“计算机、软件、出租车、地铁”等词汇。文本中提到的“点评”可能指的是大众点评，但“点评”本身可能不算特有词汇，而“手机”是常见的，但可能不算特有。这里没有出现标准中的特有词汇，所以可能得0分。\n\n2. 大陆语法习惯：是否有“挺...的、蛮...的、咋样”等。文本中“真是仅对了诶”中的“诶”可能属于口语，但“挺...的”或“蛮...的”没有出现。可能得0分。\n\n3. 大陆口语表达：是否有“搞定、整、弄”等。文本中没有这些词，所以可能得0分。\n\n4. 避免繁体用语：检查是否有“电脑、软件、资料”等繁体词。文本中没有这些词，所以得1分。\n\n5. 整体大陆化程度：综合评估。文本中的“点评”可能指大众点评，但整体用词比较

...

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-e5e6f7be4fea48cba0d7f24548943320",
  "created": 1757380270,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本中的大陆用语特征，并按照给定的五个标准进行评分，每项0或1分。首先，我要仔细阅读文本，然后逐一检查每个标准。\n\n文本是：“根据飞行中的营养标准，组合体阶段每名航天员每天需要的热量‘折合’成食物的分量，相当于一到两公斤。”\n\n首先看第一个标准：大陆特有词汇。用户给出的例子包括计算机、软件、出租车、地铁等。我需要检查文本中是否有这些词汇。文本中的“组合体阶段”可能指的是空间站的组合体，但“组合体”是否是大陆特有的词汇呢？可能不是特别常见，但不确定。另外，“折合”这个词在大陆使用较多吗？可能属于常见词汇，但不确定是否属于特有。其他词汇如“航天员”是大陆常用的，而“营养标准”可能比较通用。可能没有明显的大陆特有词汇，所以这一项可能得0分。\n\n接下来是大陆语法习惯，比如“挺...的”、“蛮...的”、“咋样”等。文本中没有这些结构，句子结构比较正式，没有使用这些习惯用法，所以这一项也是0分。\n\n第三项是大陆口语表达，比如“搞定、整、弄等”。文本中的“折合”是书面语，没有使用这些口语词，所以这一项也是0分。\n\n第四项是避免繁体用语，检查是否有“电脑、软件、资料”等。文本中没有这些词，用的是简体字，所以这一项得1分。\n

非同步批次推論:  80%|████████  | 4/5 [01:05<00:16, 16.30s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-9a5843d9fe484724847ffe8ebe9e2633",
  "created": 1757380286,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估给定的文本是否具有大陆用语的特征，并按照五个标准进行评分，每项0或1分。首先，我要仔细阅读文本，然后逐一对照每个评分标准。\n\n文本内容是：“莆田市在全市机关中开展‘读书、荐书、品书’活动，利用农家书屋在广大乡村开展以‘我的书屋、我的梦’为主题的阅读活动，开设‘夫妻课堂’‘婆媳课堂’‘亲子课堂’‘祖辈家长课堂’，等等。”\n\n首先看第一个标准：大陆特有词汇。需要检查是否有计算机、软件、出租车、地铁等词汇。文本中提到的“农家书屋”是大陆特有的吗？可能属于，但不确定。其他如“读书、荐书、品书”这些活动名称，可能比较常见，但可能不算特有词汇。没有出现标准中提到的那些具体词汇，所以可能得0分。\n\n第二个标准是大陆语法习惯，比如“挺...的”、“蛮...的”、“咋样”等。文本中没有这些结构，比如“挺好的”或者“咋样”之类的，所以可能得0分。\n\n第三个是大陆口语表达，如“搞定、整、弄”等。文本中没有这些词，都是比较正式的表达，所以可能得0分。\n\n第四个是避免繁体用语，比如不含“电脑、软件、资料”等。文本中的“书屋”、“课堂”都是简体字，没有繁体字，所以符合要求，得1分。\n\n第五个是整体大陆化程度，综合评估。文本中的“农家书屋”和“机

非同步批次推論: 100%|██████████| 5/5 [01:21<00:00, 16.26s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-e5d4f521f61549658c1df691e8535934",
  "created": 1757380303,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陸用語特徵，按照给定的五个评分标准，每项0或1分。首先，我需要仔细阅读文本，然后逐一检查每个标准。\n\n文本内容是：“和老公两个人怎么吃也超不过100大米由于经常去，和几个服务员都混的比较熟了，其实他们的服务态度还真是挺好的，就是服务员的数量太少啦所以有时候忙不过来。”\n\n首先看第一项：大陸特有詞彙。需要检查是否有计算机、软件、出租车、地铁等词汇。文本中提到的“大米”可能是指“米饭”，但“大米”本身是普通词汇，不是大陆特有的。其他词汇如“老公”是常见称呼，但可能不算特有词汇。这里没有出现标准中列出的特有词汇，所以可能得0分。\n\n第二项：大陸語法習慣，比如“挺...的”、“蠻...的”、“咋樣”等。文本中用了“挺好的”，符合“挺...的”的结构，所以这里应该给1分。\n\n第三项：大陸口語表達，如“搞定、整、弄”等。文本中没有出现这些词，所以可能得0分。\n\n第四项：避免繁體用語，检查是否有“電腦、軟體、資料”等。文本中没有这些词，所以得1分。\n\n第五项：整體大陸化程度，综合评估。文本中使用了“老公”、“挺好的”等，但整体可能不算特别大陆化，但可能还是给1分？或者可能因为其他因素给0？需要再考虑。\n\n总分

## Qwen-512

MODEL_NAME = "Qwen3-30B-A3B"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

In [14]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio



# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }


async def mainland_score_api_async(text, session, model_endpoint, api_key, model_name):
    """使用你提供的 API 非同步推論大陸用語分數"""

    prompt = f"""
你是語言檢測工具，請針對下列文本按五項標準打分，每項為 0 或 1，總分為 0~5。
禁止說明理由，或任何其他解釋，請使用以下格式回傳：

大陸特有詞彙:x
大陸語法習慣:x
大陸口語表達:x
避免繁體用語:x
整體大陸化程度:x
總分:x

文本：{text}
"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 512
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            if response.status != 200:
                return f"[ERROR] API HTTP 狀態碼: {response.status}"
            
            data = await response.json()
            
            # 加這個，看看整包回傳長怎樣
            return f"[DEBUG RAW RESPONSE]\n{json.dumps(data, indent=2, ensure_ascii=False)}"

    except Exception as e:
        return f"[EXCEPTION] {str(e)}"


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="Qwen3-30B-A3B",
                                        text_col='text', sample_size=100, threshold=3, batch_size=20):
    
    
    print(f"📊 處理資料集：{len(df)} 筆")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    indices = sample_df.index.tolist()

    results = []
    authentic_texts = []
    generic_texts = []

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)
            for i, response in enumerate(responses):
                print(f"DEBUG response {i}:\n{response}\n{'-'*40}")

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length']
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "Qwen3-30B-A3B"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 3870 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：3870 筆


非同步批次推論:  20%|██        | 1/5 [00:16<01:06, 16.59s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-bba65622156b4558b4917e2e21acb815",
  "created": 1757380447,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要处理用户的这个请求，作为语言检测工具，对给定的文本按照五个标准打分，每个标准是0或1，总分0-5。用户还特别强调不要解释理由，直接按格式返回结果。\n\n首先，我需要仔细阅读用户提供的文本，然后逐一分析每个标准。让我先通读一遍文本：\n\n“现在已经成了吃小海鲜就会想到的地方因为去旁边的锅一烧而路过的地方，本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了一跳，还真是便宜的说现在已经吃遍了这里所有的菜和主食，写下自己不喜欢的，剩下的都OK：卤水蚍蛴香螺因为我比较喜欢吃酱爆口味的。”\n\n接下来，我需要检查每个标准：\n\n1. 大陆特有词汇：比如“小海鲜”、“锅一烧”、“倪式海泰”、“卤水蚍蛴香螺”这些词汇是否属于大陆特有的。可能“锅一烧”是某种餐厅或菜名，但不确定是否大陆特有。“倪式海泰”可能是一个品牌，但不确定是否大陆的。另外，“蚍蛴”可能是指某种海鲜，但可能用的是简体字，不过不确定是否大陆特有。可能需要打1分，但不确定。\n\n2. 大陆语法习惯：句子结构是否符合大陆的语法。比如“因为去旁边的锅一烧而路过的地方”可能语法不太通顺，但可能符合口语化表达。可能打1分，但不确定。\n\n3. 大陆口语表达：比如“被菜单吓了一跳”、“剩下的

非同步批次推論:  40%|████      | 2/5 [00:32<00:48, 16.12s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-22481008ce844638881dba55c7ec5c92",
  "created": 1757380463,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\n大陸特有詞彙:x1\n大陸語法習慣:x1\n大陸口語表達:x1\n避免繁體用語:x1\n整體大陸化程度:x1\n總分:x5",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要处理用户的这个请求，作为语言检测工具，对给定的文本按照五个标准打分，每个标准是0或1，总分0-5。首先，我得仔细理解每个评分标准的含义。\n\n用户提供的文本是：“一开始不知道这里是吃什么的，很少来这里消费，现拿手机上点评查的，哈哈，后来进来之后就觉得真是仅对了诶，没想到这里有拉面炒年糕，哈哈，一直想在北京找一家正宗的拉面炒年糕店，于是点了试吃。”\n\n首先，我需要检查每个标准：\n\n1. 大陆特有词汇：比如“点评”、“试吃”、“拉面炒年糕”这些词是否是大陆特有的。比如“点评”可能指的是大众点评，这是大陆常见的应用，所以可能算1分。“试吃”也是常见词汇，可能算1分。但不确定是否足够特有，可能需要再考虑。\n\n2. 大陆语法习惯：比如句子结构是否符合大陆的表达方式。例如“一开始不知道这里是吃什么的”这样的结构是否常见。可能没问题，所以可能给1分。\n\n3. 大陆口语表达：比如“哈哈”、“诶”这些语气词是否常见。大陆口语中常用这些，可能给1分。\n\n4. 避免繁体用语：文本中没有繁体字，都是简体字，所以应该给1分。\n

...

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-f4333dbb80cc498aba56d91c9d4f3d26",
  "created": 1757380479,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要处理用户的这个请求，作为语言检测工具，对给定的文本按照五个标准打分，每个标准是0或1，总分0-5。用户特别强调不要解释理由，只要按照指定格式返回结果。首先，我得仔细分析文本，然后逐一检查每个标准。\n\n首先看文本内容：“根据飞行中的营养标准，组合体阶段每名航天员每天需要的热量‘折合’成食物的分量，相当于一到两公斤。” 这里的关键词可能有“组合体阶段”、“航天员”、“折合”等。接下来我需要逐项分析。\n\n第一项是“大陸特有詞彙”。需要判断是否有大陆特有的词汇。比如“组合体阶段”可能是指空间站的组合体阶段，这在大陆的航天术语中可能常用。另外“航天员”是大陆的用法，台湾通常用“太空人”。还有“折合”这个词，可能在大陆的语境中更常见。所以可能这里有几个大陆特有的词汇，所以可能给1分。\n\n第二项是“大陸語法習慣”。需要看句子结构是否符合大陆的语法习惯。比如“根据...标准，...需要的热量‘折合’成食物的分量”这样的结构，可能比较符合大陆的书面语习惯，没有明显的语法错误，所以可能给1分。\n\n第三项是“大陸口語表達”。这里文本看起来比较正式，属于科技或官方文本，可能没有明显的口语表达，所以可能给0分。\n\n第四项是“避免繁體用語”。文本中没有

非同步批次推論:  80%|████████  | 4/5 [01:04<00:16, 16.02s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-d70bb07171b849e3aa261898fe881ecd",
  "created": 1757380495,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\n大陸特有詞彙:x\n大陸語法習慣:x\n大陸口語表達:x\n避免繁體用語:x\n整體大陸化程度:x\n總分:x",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要处理用户的这个请求，作为语言检测工具，对给定的文本按照五个标准打分，每个标准是0或1，总分0-5。用户还特别强调不要解释理由，直接按格式返回结果。首先，我得仔细分析文本，然后逐一检查每个标准。\n\n首先看“大陸特有詞彙”这个标准。文本中有没有大陆特有的词汇呢？比如“农家书屋”、“夫妻课堂”这些可能比较有中国特色的词汇，但不确定是否属于特有词汇。可能需要判断这些是否在台湾或其他地区也使用。不过“农家书屋”听起来像是中国大陆特有的公共阅读设施，所以可能算1分。\n\n接下来是“大陸語法習慣”。需要检查句子结构是否符合大陆的语法习惯。比如“在全市机关中开展...活动”这样的结构，可能比较常见于大陆的官方文件或新闻报道，所以可能给1分。\n\n第三项是“大陸口語表達”。文本中的表达是否口语化？比如“读书、荐书、品书”这样的短语可能比较书面化，但“夫妻课堂”等可能更偏向于口语或宣传用语。不过整体来看，可能更偏向书面语，所以可能给0分。\n\n第四项是“避免繁體用語”。文本中是否有繁体字？用户提供的文本是简体中文，所以应该没有繁体字，因此给1分。

非同步批次推論: 100%|██████████| 5/5 [01:20<00:00, 16.06s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-0cbd9328de2a421188f0f633d0e6a073",
  "created": 1757380511,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要处理用户的这个请求，作为语言检测工具，对给定的文本按照五个标准打分，每个标准是0或1，总分0-5。用户还特别强调不要解释理由，直接按格式返回结果。\n\n首先，我需要仔细分析用户提供的文本：“和老公两个人怎么吃也超不过100大米由于经常去，和几个服务员都混的比较熟了，其实他们的服务态度还真是挺好的，就是服务员的数量太少啦所以有时候忙不过来。”\n\n接下来，我需要逐项检查每个标准：\n\n1. 大陆特有词汇：看看有没有大陆特有的词汇。比如“大米”可能指的是米饭，但“大米”本身在大陆和台湾都可能使用，不过“超不过”可能更口语化，但不确定是否属于大陆特有。另外“混的比较熟”中的“混”可能比较口语，但可能不算特有词汇。可能这里没有明显的大陆特有词汇，所以可能得0分？\n\n2. 大陆语法习惯：检查语法是否符合大陆习惯。比如“和老公两个人怎么吃也超不过100大米”这里的结构是否正确？“超不过”可能更常见的是“吃不完”或者“不够吃”，但“超不过”可能不太常见，可能有语法问题？或者是否符合大陆的表达方式？可能这里语法有些问题，但不确定是否属于大陆语法习惯。可能得0分？\n\n3. 大陆口语表达：比如“混的比较熟”、“服务态度还真是挺好的”、“忙不过来”这些

In [16]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio



# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }


async def mainland_score_api_async(text, session, model_endpoint, api_key, model_name):
    """使用你提供的 API 非同步推論大陸用語分數"""

    prompt = f"""
請評估以下文本的大陸用語特徵，並直接以式輸出評分結果。評分標準為每項 0 或 1 分，其中「總分」為所有分數加總。

文本：{text}

評分標準：
1. 大陸特有詞彙：文本中是否包含字典詞彙，如「計算機」、「軟件」、「質量」、「打車」。
2. 大陸語法習慣：文本中是否包含常見語法，如「挺...的」、「蠻...的」、「啥樣」。
3. 大陸口語表達：文本中是否包含口語詞彙，如「搞定」、「整」、「弄」、「咋」、「立馬」。
4. 避免繁體用語：文本中是否不包含繁體用語，如「電腦」、「軟體」、「資料」、「計程車」。
5. 整體大陸化程度：綜合評估文本的整體風格和用詞習慣。

請注意，你的回答必須簡潔，不要有任何多餘的解釋或引言。直接以以下格式開始你的回答：

結論：[你對文本風格的總結]
大陸特有詞彙: [分數]
大陸語法習慣: [分數]
大陸口語表達: [分數]
避免繁體用語: [分數]
整體大陸化程度: [分數]
總分: [總分]
"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 512
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            if response.status != 200:
                return f"[ERROR] API HTTP 狀態碼: {response.status}"
            
            data = await response.json()
            
            # 加這個，看看整包回傳長怎樣
            return f"[DEBUG RAW RESPONSE]\n{json.dumps(data, indent=2, ensure_ascii=False)}"

    except Exception as e:
        return f"[EXCEPTION] {str(e)}"


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="Qwen3-30B-A3B",
                                        text_col='text', sample_size=100, threshold=3, batch_size=20):
    
    
    print(f"📊 處理資料集：{len(df)} 筆")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    indices = sample_df.index.tolist()

    results = []
    authentic_texts = []
    generic_texts = []

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)
            for i, response in enumerate(responses):
                print(f"DEBUG response {i}:\n{response}\n{'-'*40}")

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length']
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "Qwen3-30B-A3B"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 3870 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：3870 筆


非同步批次推論:  20%|██        | 1/5 [00:17<01:08, 17.10s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-0f1d132601ff47a8a1a7f545642d365a",
  "created": 1757380748,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陆用语特征，并按照给定的五个标准进行评分，每项0或1分，最后总分是各分之和。首先，我需要仔细阅读文本，然后逐一检查每个标准。\n\n首先看文本内容：“现在已经成了吃小海鲜就会想到的地方因为去旁边的锅一烧而路过的地方，本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了一跳，还真是便宜的说现在已经吃遍了这里所有的菜和主食，写下自己不喜欢的，剩下的都OK：卤水蚍蛴香螺因为我比较喜欢吃酱爆口味的。”\n\n接下来，按照五个标准逐一分析：\n\n1. 大陆特有词汇：需要检查是否有“计算机”、“软件”、“质量”、“打车”等词。文本中没有这些词，出现的如“小海鲜”、“锅一烧”可能不是标准大陆特有词汇，所以可能得0分。\n\n2. 大陆语法习惯：是否有“挺...的”、“蛮...的”、“啥样”等。文本中没有这些结构，比如“挺贵的”或“蛮便宜的”，但原文有“便宜的说”，可能不算标准结构，所以可能0分。\n\n3. 大陆口语表达：是否有“搞定”、“整”、“弄”、“咋”、“立马”等。文本中没有这些词，但有“OK”可能算口语，但不确定是否符合标准中的词汇，可能0分。\n\n4. 避免繁体用语：检查是否有“电脑”、“软件”、“资料”、“计程车”等

非同步批次推論:  40%|████      | 2/5 [00:34<00:51, 17.05s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-d01b5e546dac44fcb8a9df652a9dd13e",
  "created": 1757380765,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陆用语特征，并按照给定的五个标准进行评分，每个标准0或1分，最后总分是各分之和。首先，我需要仔细阅读文本，然后逐一检查每个标准。\n\n文本内容是：“一开始不知道这里是吃什么的，很少来这里消费，现拿手机上点评查的，哈哈，后来进来之后就觉得真是仅对了诶，没想到这里有拉面炒年糕，哈哈，一直想在北京找一家正宗的拉面炒年糕店，于是点了试吃。”\n\n首先看第一个标准：大陆特有词汇。需要检查是否有“计算机”、“软件”、“质量”、“打车”这些词。文本中没有这些词，所以这里应该是0分。\n\n第二个标准：大陆语法习惯，比如“挺...的”、“蛮...的”、“啥样”。文本中没有这些结构，比如“挺好的”或“蛮厉害的”，所以可能也是0分。\n\n第三个标准：大陆口语表达，如“搞定”、“整”、“弄”、“咋”、“立马”。文本中用了“哈哈”、“诶”这些语气词，但“搞定”之类的词没有出现，所以可能0分。不过“试吃”可能有点口语，但不确定是否属于指定词汇，可能还是0分。\n\n第四个标准：避免繁体用语，比如“电脑”、“软件”、“资料”、“计程车”。文本中没有这些繁体字，所以应该是1分，因为用的是简体字。\n\n第五个标准：整体大陆化程度。整体来看，文本用

...

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-350b47430f7f4029aeaeb67b984e1315",
  "created": 1757380781,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陆用语特征，并按照给定的五个标准进行评分，每个标准满分1分，总分是各分项的总和。首先，我需要仔细阅读用户提供的文本，然后逐一检查每个评分标准。\n\n首先看文本内容：“根据飞行中的营养标准，组合体阶段每名航天员每天需要的热量‘折合’成食物的分量，相当于一到两公斤。”\n\n接下来，我需要检查每个评分标准：\n\n1. 大陆特有词汇：需要看是否有“计算机”、“软件”、“质量”、“打车”这些词。文本中没有出现这些词，所以这里可能得0分。不过要注意是否有其他大陆特有的词汇，比如“组合体”可能是指空间站的组合体阶段，但不确定是否属于特有词汇。不过根据用户给的列表，这里没有出现，所以这一项应该是0分。\n\n2. 大陆语法习惯：检查是否有“挺...的”、“蛮...的”、“啥样”等结构。文本中的句子结构比较正式，没有这些语法结构，所以这一项应该是0分。\n\n3. 大陆口语表达：是否有“搞定”、“整”、“弄”、“咋”、“立马”等词。文本中使用的是“折合”、“相当于”等正式用语，没有口语词汇，所以这里也是0分。\n\n4. 避免繁体用语：检查是否有“电脑”、“软件”、“资料”、“计程车”等繁体词。文本中没有这些词，用的是简体字，所以这一

非同步批次推論:  80%|████████  | 4/5 [01:06<00:16, 16.54s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-9854c774864341f49adc3755d6d64f68",
  "created": 1757380798,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陆用语特征，并按照给定的五个标准进行评分，每个标准满分1分，总分5分。首先，我要仔细阅读用户给的文本，然后逐一检查每个评分标准。\n\n首先看文本内容：“莆田市在全市机关中开展‘读书、荐书、品书’活动，利用农家书屋在广大乡村开展以‘我的书屋、我的梦’为主题的阅读活动，开设‘夫妻课堂’‘婆媳课堂’‘亲子课堂’‘祖辈家长课堂’，等等。”\n\n接下来，按照五个标准来分析：\n\n1. 大陆特有词汇：需要检查是否有如“计算机”、“软件”、“质量”、“打车”等词汇。文本中出现的“农家书屋”、“读书、荐书、品书”这些可能属于大陆特有的说法，但用户给的例子中并没有直接出现那些特定词汇。比如“农家书屋”是大陆常见的，但不确定是否属于字典词汇中的特有词。可能这里没有直接出现标准中的例子，所以可能得0分。\n\n2. 大陆语法习惯：检查是否有“挺...的”、“蛮...的”、“啥样”等。文本中没有这些结构，比如“挺好的”或者“蛮厉害的”，所以可能得0分。\n\n3. 大陆口语表达：是否有“搞定”、“整”、“弄”、“咋”、“立马”等。文本中的“开展”、“利用”、“开设”都是比较正式的词汇，没有口语化的词，所以可能得0分。\n\n4. 避免繁体用

非同步批次推論: 100%|██████████| 5/5 [01:23<00:00, 16.61s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-087362be5bdc4122a72fff885cda654a",
  "created": 1757380814,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\n\n結論：文本風格偏向口語化，使用部分大陸常用表達但缺乏明顯特有詞彙\n大陸特有詞彙: 0\n大陸語法習慣: 1\n大陸口語表達: 0\n避免繁體用語: 1\n整體大陸化程度: 1\n總分: 3",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陸用語特徵，并按照给定的五个标准进行评分，每个标准0或1分，最后总分是各分數的總和。首先，我需要仔细阅读文本，然后逐一检查每个标准。\n\n文本内容是：“和老公两个人怎么吃也超不过100大米由于经常去，和几个服务员都混的比较熟了，其实他们的服务态度还真是挺好的，就是服务员的数量太少啦所以有时候忙不过来。”\n\n首先看第一项：大陸特有詞彙。需要检查是否有“計算機”、“軟件”、“質量”、“打車”这些词。文本中没有这些词，所以这里应该是0分。\n\n第二项：大陸語法習慣。常见的语法如“挺...的”、“蠻...的”、“啥樣”。文本中出现了“挺好的”，符合“挺...的”结构，所以这里应该给1分。\n\n第三项：大陸口語表達。检查是否有“搞定”、“整”、“弄”、“咋”、“立馬”等词。文本中没有这些词，但有“超不過”、“混的比較熟了”、“忙不過來”等，不过这些可能不算标准的口语词汇。所以这里可能0分。\n\n第四项：避免繁體用語。检查是否有“電腦”

In [19]:
MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"  
OPENWEBUI_API_KEY = "sk-b56c488f33b94df297a6314bd037b805"  
MODEL_NAME = "Qwen3-30B-A3B" 
text_column = 'text'

In [21]:
test_text = "我在計算機上安裝了軟件，挺好用的。"
async with aiohttp.ClientSession() as session:
    reply = await mainland_score_api_async(test_text, session, MODEL_API_ENDPOINT, OPENWEBUI_API_KEY,MODEL_NAME)
    print("🔹 最終 reply:", reply)


🔹 最終 reply: [DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-41c4d4a22a49450994c465443a71575a",
  "created": 1757381306,
  "model": "hosted_vllm//root/Qwen3-30B-A3B",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": null,
        "role": "assistant",
        "tool_calls": null,
        "function_call": null,
        "reasoning_content": "\n好的，我现在需要评估用户提供的文本的大陆用语特征，并按照给定的评分标准给出每个项目的分数。首先，我需要仔细阅读用户的要求，确保自己完全理解每个评分项的定义。\n\n首先看文本：“我在計算機上安裝了軟件，挺好用的。” 这里的关键词是“計算機”和“軟件”。根据评分标准的第一项，大陆特有词汇包括“計算機”、“軟件”、“質量”、“打車”。这里“計算機”对应的是“电脑”，而“軟件”对应“软件”，但要注意的是，大陆通常使用“计算机”和“软件”，而繁体字中可能用“計算機”和“軟體”。不过这里用户提到的是大陆用语，所以可能需要看是否使用简体字还是繁体字。但文本中的“計算機”和“軟件”是繁体字，但用户可能是在测试用词而非字形。不过评分标准中的“大陸特有詞彙”可能指的是词汇本身，而不是字形。例如，“計算機”在大陆是“计算机”，而“軟件”是“软件”，所以这两个词在大陆是正确的，但可能用户是否使用繁体字的问题？\n\n不过用户的问题可能更关注词汇本身是否属于大陆常用，而不是字形。例如，“計算機”在大陆是“计算机”，而“軟件”是“软件”，所以这两个词在大陆是正确的，但可能用户是否使用繁体字的问题。不过评分标准中的“大陸特有詞彙”可能指的是词汇，而不是字形。例如，“計算機”在大陆是“计算机”，而“軟

In [ ]:
test_text = "我在計算機上安裝了軟件，挺好用的。"
async with aiohttp.ClientSession() as session:
    reply = await mainland_score_api_async(test_text, session, MODEL_API_ENDPOINT, OPENWEBUI_API_KEY,MODEL_NAME)
    scores = parse_scores(reply)
    print("測試文本分數:", scores)

測試文本分數: {'大陸特有詞彙': 0, '大陸語法習慣': 0, '大陸口語表達': 0, '避免繁體用語': 0, '整體大陸化程度': 0, '總分': 0}


In [ ]:
# original_text =split_dataset_df.loc[690, 'text']
    
# original_text = str(original_text)

# # 擷取原始文本中的片段
# extracted_text = original_text[24:63]

# print(f"原始文本: {original_text}")
# print(f"擷取的片段: {extracted_text}")


## gemma- 100

In [24]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio



# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }


async def mainland_score_api_async(text, session, model_endpoint, api_key, model_name):
    """使用你提供的 API 非同步推論大陸用語分數"""

    prompt = f"""評估文本的大陸用語特徵，每項0或1分：

文本：{text}

評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 100
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            data = await response.json()
            reply = data.get("choices", [{}])[0].get("message", {}).get("content", None)
            return reply
    except Exception as e:
        return str(e)


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="gemma-3-27b-it",
                                        text_col='text', sample_size=100, threshold=3, batch_size=20):
    print(f"📊 處理資料集：{len(df)} 筆")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    indices = sample_df.index.tolist()

    results = []
    authentic_texts = []
    generic_texts = []

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length']
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "gemma-3-27b-it"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 3870 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：3870 筆


非同步批次推論: 100%|██████████| 5/5 [00:15<00:00,  3.18s/it]


📊 篩選結果統計:
  ✅ 真正大陸用語: 13 筆
  🗑️ 通用簡體中文: 87 筆
  📈 篩選率: 13.0%

📝 高質量大陸用語範例:
  1. (得分:3) 服务员的态度我觉得也挺好的，人多，上菜慢，很多桌都有催的，但她们不是在顾客面前应付一下，转身干自己的事，而是真正的帮每桌...
  2. (得分:3) 我们整个企业今天到现在为止总计接单一共六十几万对， 其中布鞋（冷粘工艺）包括布配皮合计25万对，店内搜索页-热风男鞋旗舰...
  3. (得分:3) 第一次玩桌面游戏还是很新奇的三国杀有点难度对于我们这些初学者来说 呵呵周日和同事约好在大世界的藏宝海湾ME是第一个到了没...

💾 儲存結果...
💾 儲存完成:
  📄 完整結果: mainland_filtering_complete_20250908_203004.csv
  ✅ 高質量句子片段數據: authentic_mainland_texts_20250908_203004.csv
  📋 JSON格式: authentic_mainland_texts_20250908_203004.json

🎉 大陸用語識別與篩選完成！
📋 可用變數: mainland_filtering_results, authentic_mainland_data
🎯 最終輸出為句子級別的片段資料 (10-50字)


In [25]:
import nest_asyncio
nest_asyncio.apply()



# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }

import aiohttp
import asyncio

async def mainland_score_api_async(text, session, model_endpoint, api_key=API_KEY, model_name="Qwen3-30B-A3B"):
    """使用你提供的 API 非同步推論大陸用語分數"""

    prompt = f"""
你是語言檢測工具，請專業的針對下列文本按五項標準打分，每項為 0 或 1，總分為 0~5。
評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估
下面是要判斷的文本
文本：{text}

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 100
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            data = await response.json()
            reply = data.get("choices", [{}])[0].get("message", {}).get("content", None)
            return reply
    except Exception as e:
        return str(e)


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="Qwen3-30B-A3B",
                                        text_col='text', sample_size=100, threshold=3, batch_size=20):
    print(f"📊 處理資料集：{len(df)} 筆")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    indices = sample_df.index.tolist()

    results = []
    authentic_texts = []
    generic_texts = []

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length']
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "gemma-3-27b-it"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 3870 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：3870 筆


非同步批次推論: 100%|██████████| 5/5 [00:15<00:00,  3.17s/it]


📊 篩選結果統計:
  ✅ 真正大陸用語: 17 筆
  🗑️ 通用簡體中文: 83 筆
  📈 篩選率: 17.0%

📝 高質量大陸用語範例:
  1. (得分:3) 现在已经成了吃小海鲜就会想到的地方因为去旁边的锅一烧而路过的地方，本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了...
  2. (得分:3) 服务员的态度我觉得也挺好的，人多，上菜慢，很多桌都有催的，但她们不是在顾客面前应付一下，转身干自己的事，而是真正的帮每桌...
  3. (得分:3) 4 开始学生肯定不多，慢慢来，口碑好了，不用宣传都有人找上门来。

💾 儲存結果...
💾 儲存完成:
  📄 完整結果: mainland_filtering_complete_20250908_203424.csv
  ✅ 高質量句子片段數據: authentic_mainland_texts_20250908_203424.csv
  📋 JSON格式: authentic_mainland_texts_20250908_203424.json

🎉 大陸用語識別與篩選完成！
📋 可用變數: mainland_filtering_results, authentic_mainland_data
🎯 最終輸出為句子級別的片段資料 (10-50字)


In [26]:
import nest_asyncio
nest_asyncio.apply()



# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }

import aiohttp
import asyncio

async def mainland_score_api_async(text, session, model_endpoint, api_key=API_KEY, model_name="Qwen3-30B-A3B"):
    """使用你提供的 API 非同步推論大陸用語分數"""

    prompt = f"""
你是語言檢測工具，請專業的針對下列文本按五項標準打分，每項為 0 或 1，總分為 0~5。
評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估
下面是要判斷的文本
文本：{text}

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 100
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            data = await response.json()
            reply = data.get("choices", [{}])[0].get("message", {}).get("content", None)
            return reply
    except Exception as e:
        return str(e)


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="Qwen3-30B-A3B",
                                        text_col='text', sample_size=100, threshold=3, batch_size=20):
    print(f"📊 處理資料集：{len(df)} 筆")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    indices = sample_df.index.tolist()

    results = []
    authentic_texts = []
    generic_texts = []

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length']
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "gemma-3-27b-it"
    SAMPLE_SIZE =   200
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 3870 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：3870 筆


非同步批次推論: 100%|██████████| 10/10 [00:21<00:00,  2.17s/it]


📊 篩選結果統計:
  ✅ 真正大陸用語: 30 筆
  🗑️ 通用簡體中文: 170 筆
  📈 篩選率: 15.0%

📝 高質量大陸用語範例:
  1. (得分:3) 现在已经成了吃小海鲜就会想到的地方因为去旁边的锅一烧而路过的地方，本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了...
  2. (得分:3) 服务员的态度我觉得也挺好的，人多，上菜慢，很多桌都有催的，但她们不是在顾客面前应付一下，转身干自己的事，而是真正的帮每桌...
  3. (得分:3) 4 开始学生肯定不多，慢慢来，口碑好了，不用宣传都有人找上门来。

💾 儲存結果...
💾 儲存完成:
  📄 完整結果: mainland_filtering_complete_20250908_203654.csv
  ✅ 高質量句子片段數據: authentic_mainland_texts_20250908_203654.csv
  📋 JSON格式: authentic_mainland_texts_20250908_203654.json

🎉 大陸用語識別與篩選完成！
📋 可用變數: mainland_filtering_results, authentic_mainland_data
🎯 最終輸出為句子級別的片段資料 (10-50字)


In [37]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio



# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }


async def mainland_score_api_async(text, session, model_endpoint, api_key, model_name):
    

    prompt = f"""
你是語言檢測工具，請專業的針對下列文本按五項標準打分，每項為 0 或 1，總分為 0~5。
評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估
下面是要判斷的文本
文本：{text}

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""


    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 512
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            if response.status != 200:
                return f"[ERROR] API HTTP 狀態碼: {response.status}"
            
            data = await response.json()
            
            # 加這個，看看整包回傳長怎樣
            return f"[DEBUG RAW RESPONSE]\n{json.dumps(data, indent=2, ensure_ascii=False)}"

    except Exception as e:
        return f"[EXCEPTION] {str(e)}"


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="Qwen3-30B-A3B",
                                        text_col='text', sample_size=100, threshold=3, batch_size=20):
    
    
    print(f"📊 處理資料集：{len(df)} 筆")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    indices = sample_df.index.tolist()

    results = []
    authentic_texts = []
    generic_texts = []

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)
            for i, response in enumerate(responses):
                print(f"DEBUG response {i}:\n{response}\n{'-'*40}")

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                # 找到切割文本
                original_row = available_data.iloc[idx]
                start_position = original_row.get('fragment_start', -1)
                end_position = original_row.get('fragment_end', -1)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'start_position': start_position,  # 新增字串的起始位置
                    'end_position': end_position,  # 新增字串的結束位置
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'index': r['index'],  # 儲存索引
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms']),
            'start_position': r['start_position'],  # 儲存起始位置
            'end_position': r['end_position']  # 儲存結束位置
        }
        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length'],
                'start_position': r['start_position'],  # 加入起始位置
                'end_position': r['end_position']  # 加入結束位置
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "gemma-3-27b-it"
    SAMPLE_SIZE = 200
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 3870 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：3870 筆


非同步批次推論:  10%|█         | 1/10 [00:07<01:03,  7.06s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-464eb722-0a43-46d9-9931-b6021d191c81",
  "created": 1757385347,
  "model": "hosted_vllm/gemma-3-27b-it",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "好的，我將針對您提供的文本進行評分，並按照指定格式回答：\n\n大陸特有詞彙:0\n大陸語法習慣:0\n大陸口語表達:1\n避免繁體用語:1\n整體大陸化程度:1\n總分:3\n\n**說明：**\n\n*   **大陸特有詞彙:** 文本中沒有明顯的大陸特有詞彙，例如“計算機”、“軟件”、“出租車”、“地鐵”等。\n*   **大陸語法習慣:** 文本中沒有“挺...的”、“蠻...的”、“咋樣”等大陸語法習慣。\n*   **大陸口語表達:** “弄”字出現，屬於大陸口語表達。\n*   **避免繁體用語:** 文本中沒有使用“電腦”、“軟體”、“資料”等繁體用語。\n*   **整體大陸化程度:** 綜合考量，文本整體大陸化程度較低，但“弄”字的出現略微提升了大陸化程度。\n",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null
      },
      "provider_specific_fields": {
        "stop_reason": 106
      }
    }
  ],
  "usage": {
    "completion_tokens": 218,
    "prompt_tokens": 333,
    "total_tokens": 551,
    "completion_tokens_details": null,
    "pr

非同步批次推論:  20%|██        | 2/10 [00:12<00:48,  6.00s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-795ba35b-05c4-454f-a00e-1d73249c61b9",
  "created": 1757385353,
  "model": "hosted_vllm/gemma-3-27b-it",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "好的，針對您提供的文本進行評分如下：\n\n大陸特有詞彙:0\n大陸語法習慣:1 (例如：挺...的，雖然文本中沒有直接出現，但語氣和表達方式接近)\n大陸口語表達:1 (例如：搞定、整、弄等，文本中出現“哈哈”的使用頻率較高，符合大陸網路用語習慣)\n避免繁體用語:1\n整體大陸化程度:1\n總分:4\n\n**說明：**\n\n雖然文本中沒有明顯的大陸特有詞彙，但其語法習慣和口語表達方式，以及整體語氣，更接近大陸網路用語習慣。因此，在「大陸語法習慣」、「大陸口語表達」和「整體大陸化程度」上給予了 1 分。\n",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null
      },
      "provider_specific_fields": {
        "stop_reason": 106
      }
    }
  ],
  "usage": {
    "completion_tokens": 175,
    "prompt_tokens": 304,
    "total_tokens": 479,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "service_tier": null,
  "prompt_logprobs": null,
  "

...

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-29b0fadd-e866-48e3-97ad-29c99bee2703",
  "created": 1757385385,
  "model": "hosted_vllm/gemma-3-27b-it",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "好的，針對您提供的文本進行評分如下：\n\n大陸特有詞彙:0\n大陸語法習慣:0\n大陸口語表達:0\n避免繁體用語:1\n整體大陸化程度:0\n總分:1\n",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null
      },
      "provider_specific_fields": {
        "stop_reason": 106
      }
    }
  ],
  "usage": {
    "completion_tokens": 54,
    "prompt_tokens": 267,
    "total_tokens": 321,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "service_tier": null,
  "prompt_logprobs": null,
  "kv_transfer_params": null
}
----------------------------------------
DEBUG response 1:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-e2b30950-baf2-445f-a862-0209eb9cf800",
  "created": 1

非同步批次推論:  90%|█████████ | 9/10 [00:49<00:05,  5.21s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-988d116c-88b0-49c9-b44f-51164c0e7acf",
  "created": 1757385390,
  "model": "hosted_vllm/gemma-3-27b-it",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "好的，針對您提供的文本進行評分如下：\n\n大陸特有詞彙:0\n大陸語法習慣:0\n大陸口語表達:0\n避免繁體用語:1\n整體大陸化程度:0\n總分:1\n\n評分說明：\n\n*   **大陸特有詞彙:** 文本中沒有出現「計算機」、「軟件」、「出租車」、「地鐵」等明顯的大陸特有詞彙。\n*   **大陸語法習慣:** 文本中未使用「挺...的」、「蠻...的」、「咋樣」等大陸常見的語法習慣。\n*   **大陸口語表達:** 文本中沒有使用「搞定」、「整」、「弄」等大陸口語表達。\n*   **避免繁體用語:** 文本中未使用「電腦」、「軟體」、「資料」等繁體中文詞彙，符合要求。\n*   **整體大陸化程度:** 綜合來看，文本用語較為正式，沒有明顯的大陸化傾向。\n",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null
      },
      "provider_specific_fields": {
        "stop_reason": 106
      }
    }
  ],
  "usage": {
    "completion_tokens": 213,
    "prompt_tokens": 283,
    "total_tokens": 496,
    "completion_tokens_details": null,
    "promp

非同步批次推論: 100%|██████████| 10/10 [00:55<00:00,  5.54s/it]

DEBUG response 0:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-766a5607-11c4-4719-91d2-49b7852fda8e",
  "created": 1757385396,
  "model": "hosted_vllm/gemma-3-27b-it",
  "object": "chat.completion",
  "system_fingerprint": null,
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "好的，針對您提供的文本進行評分如下：\n\n大陸特有詞彙:0\n大陸語法習慣:0\n大陸口語表達:0\n避免繁體用語:1\n整體大陸化程度:0\n總分:1\n",
        "role": "assistant",
        "tool_calls": null,
        "function_call": null
      },
      "provider_specific_fields": {
        "stop_reason": 106
      }
    }
  ],
  "usage": {
    "completion_tokens": 54,
    "prompt_tokens": 291,
    "total_tokens": 345,
    "completion_tokens_details": null,
    "prompt_tokens_details": null
  },
  "service_tier": null,
  "prompt_logprobs": null,
  "kv_transfer_params": null
}
----------------------------------------
DEBUG response 1:
[DEBUG RAW RESPONSE]
{
  "id": "chatcmpl-39e0758a-5619-4ed7-b891-3df5702b91ce",
  "created": 1

In [27]:
MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"  
OPENWEBUI_API_KEY = "sk-b56c488f33b94df297a6314bd037b805"  
MODEL_NAME = "gemma-3-27b-it" 
text_column = 'text'

In [28]:
test_text = "我在計算機上安裝了軟件，挺好用的。"
async with aiohttp.ClientSession() as session:
    reply = await mainland_score_api_async(test_text, session, MODEL_API_ENDPOINT, OPENWEBUI_API_KEY,MODEL_NAME)
    print("🔹 最終 reply:", reply)

🔹 最終 reply: 好的，我將針對您提供的文本進行評分：

大陸特有詞彙:1 (包含「計算機」、「軟件」)
大陸語法習慣:1 (包含「挺...的」)
大陸口語表達:0
避免繁體用語:0 (應避免使用「電腦」、「軟體」等繁體用語)
整體大陸化程度:1

總分:3



In [29]:
test_text = "我在計算機上安裝了軟件，挺好用的。"
async with aiohttp.ClientSession() as session:
    reply = await mainland_score_api_async(test_text, session, MODEL_API_ENDPOINT, OPENWEBUI_API_KEY,MODEL_NAME)
    scores = parse_scores(reply)
    print("測試文本分數:", scores)

測試文本分數: {'大陸特有詞彙': 1, '大陸語法習慣': 1, '大陸口語表達': 0, '避免繁體用語': 0, '整體大陸化程度': 1, '總分': 3}


## 📁 從大文件挑選資料

從指定的大文件中隨機挑選250筆資料進行評分處理。

In [30]:
from datasets import load_dataset
import pandas as pd
import random
from datetime import datetime
import os

# 載入 Hugging Face 資料集
print("📥 從 Hugging Face 載入 `austenjs/ClueCorpusSmallDataset`...")
dataset = load_dataset("austenjs/ClueCorpusSmallDataset", split="train")

print(f"✅ 資料集載入完成，共 {len(dataset):,} 筆")

# 設定樣本數與隨機種子
target_samples = 250
random.seed(42)

# 隨機取樣
print(f"🎲 隨機挑選 {target_samples} 筆資料...")
sampled_dataset = dataset.shuffle(seed=42).select(range(target_samples))


# 轉成 DataFrame
selected_df = sampled_dataset.to_pandas()
selected_df['text_length'] = selected_df['text'].apply(len)

# 顯示基本統計
print(f"\n📈 文本長度統計:")
print(selected_df['text_length'].describe())

# 儲存資料
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = "selected_data"
os.makedirs(save_dir, exist_ok=True)

base_filename = f"{save_dir}/selected_250_samples_hf_{timestamp}"
csv_filename = f"{base_filename}.csv"
json_filename = f"{base_filename}.json"
parquet_filename = f"{base_filename}.parquet"

selected_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
selected_df.to_json(json_filename, orient='records', force_ascii=False, indent=2)
selected_df.to_parquet(parquet_filename, index=False)

print(f"\n💾 資料已儲存:")
for file in [csv_filename, json_filename, parquet_filename]:
    size_mb = os.path.getsize(file) / (1024 * 1024)
    print(f"  {os.path.basename(file)}: {size_mb:.2f} MB")

# 設定全域變數
globals()['selected_large_file_df'] = selected_df

print(f"\n🎉 HF 資料集挑選完成！共 {len(selected_df)} 筆")
print(f"📋 可用變數: selected_large_file_df")


📥 從 Hugging Face 載入 `austenjs/ClueCorpusSmallDataset`...


Loading dataset shards:   0%|          | 0/28 [00:00<?, ?it/s]

✅ 資料集載入完成，共 16,608,422 筆
🎲 隨機挑選 250 筆資料...

📈 文本長度統計:
count     250.000000
mean      285.956000
std       559.723126
min         4.000000
25%        32.000000
50%       106.500000
75%       269.500000
max      4394.000000
Name: text_length, dtype: float64

💾 資料已儲存:
  selected_250_samples_hf_20250908_205459.csv: 0.19 MB
  selected_250_samples_hf_20250908_205459.json: 0.20 MB
  selected_250_samples_hf_20250908_205459.parquet: 0.14 MB

🎉 HF 資料集挑選完成！共 250 筆
📋 可用變數: selected_large_file_df


## 🔄 處理挑選的250筆資料

對挑選的資料進行句子級別切分和大陸用語評分。

In [31]:
MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"  # 這是你的 API 端點
OPENWEBUI_API_KEY = "sk-b56c488f33b94df297a6314bd037b805"  # 替換為你的 OpenWebUI API 金鑰
MODEL_NAME = "gemma-3-27b-it"  # 你選擇的模型名稱
SAMPLE_SIZE = 100  # 隨機樣本數量
THRESHOLD = 3  # 大陸化特徵分數的閾值
BATCH_SIZE = 20  # 每次處理的批次數量
text_column = 'text'  # 指定資料集中的文本欄位名稱

In [32]:
# 🔄 處理挑選的250筆資料 - 句子切分 + 大陸用語評分
print("🚀 開始處理挑選的250筆資料...")

if 'selected_large_file_df' in globals() and selected_large_file_df is not None:
    print(f"✅ 找到挑選的資料: {len(selected_large_file_df)} 筆")
    
    # Step 1: 句子級別切分
    print(f"\n🔪 步驟1: 執行句子級別切分...")
    
    selected_split_df = process_text_splitting(
        df=selected_large_file_df,
        text_column='text',
        min_length=30,
        max_length=250
    )
    
    if not selected_split_df.empty:
        print(f"\n📊 切分結果:")
        print(f"  原始文本: {len(selected_large_file_df)} 筆")
        print(f"  切分後句子片段: {len(selected_split_df)} 筆")
        print(f"  平均每文本產生: {len(selected_split_df)/len(selected_large_file_df):.1f} 個片段")
        
        # 顯示切分範例
        print(f"\n📝 切分範例 (前3個原始文本):")
        for orig_idx in selected_split_df['original_index'].unique()[:3]:
            fragments = selected_split_df[selected_split_df['original_index'] == orig_idx]
            original_text = selected_large_file_df.iloc[orig_idx]['text']
            
            print(f"\n原始文本 #{orig_idx}: {original_text[:100]}{'...' if len(original_text) > 100 else ''}")
            print(f"切分為 {len(fragments)} 個片段:")
            
            for i, (_, row) in enumerate(fragments.iterrows()):
                print(f"  片段{i+1} ({row['fragment_length']}字): {row['text']}")
            print("-" * 80)
        
        # Step 2: 大陸用語評分
        print(f"\n🎯 步驟2: 執行大陸用語評分...")
        
        # 設定評分參數
        SAMPLE_SIZE = min(300, len(selected_split_df))  # 從切分後的片段中取樣評分
        THRESHOLD = 3  # 評分閾值
        
        print(f"📊 評分參數:")
        print(f"  評分樣本數: {SAMPLE_SIZE}")
        print(f"  篩選閾值: {THRESHOLD}/5")
        
        # 執行評分
        results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
        )
        
        # 統計結果
        print(f"\n📈 評分結果統計:")
        print(f"  總評分數: {len(results)}")
        print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
        print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
        if len(results) > 0:
            print(f"  📊 大陸用語比例: {len(authentic_results)/len(results)*100:.1f}%")
        
        # 顯示高質量範例
        if authentic_results:
            print(f"\n🏆 高質量大陸用語片段範例:")
            for i, r in enumerate(authentic_results[:5]):
                print(f"第{i+1}名 (得分:{r['scores']['總分']}/5): {r['text']}")
                if r['features']['mainland_terms']:
                    print(f"  大陸特有詞彙: {', '.join(r['features']['mainland_terms'])}")
                print("-" * 60)
        
        # 儲存最終結果
        print(f"\n💾 儲存最終結果...")
        full_df, auth_df = save_results(results, authentic_results, generic_results)
        
        # 創建完整處理結果摘要
        summary_data = {
            "processing_summary": {
                "original_samples": len(selected_large_file_df),
                "split_fragments": len(selected_split_df),
                "evaluated_fragments": len(results),
                "high_quality_mainland": len(authentic_results),
                "generic_chinese": len(generic_results),
                "mainland_percentage": len(authentic_results)/len(results)*100 if len(results) > 0 else 0
            },
            "fragment_length_stats": {
                "mean": selected_split_df['fragment_length'].mean(),
                "min": selected_split_df['fragment_length'].min(),
                "max": selected_split_df['fragment_length'].max(),
                "median": selected_split_df['fragment_length'].median()
            }
        }
        
        # 儲存處理摘要
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        summary_file = f"processing_summary_{timestamp}.json"
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary_data, f, ensure_ascii=False, indent=2)
        
        print(f"📋 處理摘要已儲存: {summary_file}")
        
        # 設定全域變數
        globals()['selected_split_df'] = selected_split_df
        globals()['selected_mainland_results'] = results
        globals()['selected_authentic_data'] = auth_df
        
        print(f"\n🎉 250筆資料完整處理流程完成！")
        print(f"📊 最終統計:")
        print(f"  原始挑選資料: {len(selected_large_file_df)} 筆")
        print(f"  句子切分後: {len(selected_split_df)} 個片段")
        print(f"  大陸用語評分: {len(results)} 個片段")
        print(f"  高質量大陸用語: {len(authentic_results)} 個片段")
        print(f"\n📋 可用變數:")
        print(f"  selected_large_file_df: 原始挑選的250筆資料")
        print(f"  selected_split_df: 句子切分後的片段資料")
        print(f"  selected_mainland_results: 大陸用語評分結果")
        print(f"  selected_authentic_data: 高質量大陸用語片段")
        
    else:
        print("❌ 句子切分失敗，無法進行後續處理")

else:
    print("❌ 沒有找到挑選的資料")
    print("💡 請先執行前面的資料挑選步驟")

print("="*80)

🚀 開始處理挑選的250筆資料...
✅ 找到挑選的資料: 250 筆

🔪 步驟1: 執行句子級別切分...
📊 開始處理文本切分...
  原始資料筆數: 250
  文本欄位: text
  句子片段長度範圍: 30-250 字


切分進度: 100%|██████████| 250/250 [00:00<00:00, 428.66it/s]



✅ 文本切分完成！
📈 切分統計:
  處理文本數: 193
  生成句子片段數: 892
  平均每文本片段數: 4.6

📏 片段長度統計:
  平均長度: 63.2 字
  最短片段: 30 字
  最長片段: 413 字
  中位數長度: 51.0 字

📊 片段長度分布:
  短片段 (10-20字): 0 個 (0.0%)
  中短片段 (20-30字): 0 個 (0.0%)
  中等片段 (30-40字): 240 個 (26.9%)
  長片段 (40-50字): 186 個 (20.9%)
  超長片段 (50-100字): 357 個 (40.0%)

📊 切分結果:
  原始文本: 250 筆
  切分後句子片段: 892 筆
  平均每文本產生: 3.6 個片段

📝 切分範例 (前3個原始文本):

原始文本 #0: 一元秒杀活动：10月29日12：00图书商城准时开启！活动介绍 为了感谢长久以来一直关注并支持我们的粉丝，机械工业出版社电工电子分社兹定于10月15日、10月22日、10月29日，在“机械工业出版社E...
切分為 2 個片段:
  片段1 (109字): 活动介绍 为了感谢长久以来一直关注并支持我们的粉丝，机械工业出版社电工电子分社兹定于10月15日、10月22日、10月29日，在“机械工业出版社E视界”官方微书城开展三期1元秒杀活动，每期提供5种畅销品种共20本图书。
  片段2 (51字): 活动书单 本期书单（10月29日12:00） 活动详解 1.点击 “E社群”→“图书商城”进入微书城。
--------------------------------------------------------------------------------

原始文本 #2: 午评：沪指收复4000点 两市个股普涨今日早盘，沪深股指小幅低开后冲高，沪指收复4000点大关。截至午间收盘，沪指报4012.97点，涨幅1.40%；深成指报13896.2点，涨幅0.72%；创业板指...
切分為 10 個片段:
  片段1 (48字): 午评：沪指收复4000点 两市个股普涨今日早盘，沪深股指小幅低开后冲高，沪指收复4000点大关。
  片段2 (42字): 分板块看，免疫治疗、卫星导航、基因概念等涨幅居前，多元金融、证券、水务

非同步批次推論: 100%|██████████| 15/15 [00:45<00:00,  3.06s/it]


📈 評分結果統計:
  總評分數: 300
  ✅ 真正大陸用語: 44 筆
  🗑️ 通用簡體中文: 256 筆
  📊 大陸用語比例: 14.7%

🏆 高質量大陸用語片段範例:
第1名 (得分:3/5): 现在已经成了吃小海鲜就会想到的地方因为去旁边的锅一烧而路过的地方，本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了一跳，还真是便宜的说现在已经吃遍了这里所有的菜和主食，写下自己不喜欢的，剩下的都OK：卤水蚍蛴香螺因为我比较喜欢吃酱爆口味的。
------------------------------------------------------------
第2名 (得分:3/5): 服务员的态度我觉得也挺好的，人多，上菜慢，很多桌都有催的，但她们不是在顾客面前应付一下，转身干自己的事，而是真正的帮每桌去催，然后回来告知几分钟会上。
  大陸特有詞彙: 挺
------------------------------------------------------------
第3名 (得分:3/5): 4 开始学生肯定不多，慢慢来，口碑好了，不用宣传都有人找上门来。
------------------------------------------------------------
第4名 (得分:3/5): 我们整个企业今天到现在为止总计接单一共六十几万对， 其中布鞋（冷粘工艺）包括布配皮合计25万对，店内搜索页-热风男鞋旗舰店 （最后一个款是我们生产的）， 软底软面类型4万对，（没有图片，懒得找） 布洛克及其变形3万对， 其他不计。
------------------------------------------------------------
第5名 (得分:5/5): 第一次玩桌面游戏还是很新奇的三国杀有点难度对于我们这些初学者来说 呵呵周日和同事约好在大世界的藏宝海湾ME是第一个到了没法子.谁叫我是号召人的LG呢 -等了20分钟大家基本来了讨论先玩啥呢其实我早在网上看过三国杀的flash比他们领先一步了当然先玩咯.但实际比我想的复杂的多我们这群笨笨半小时晕忽忽算了就换吧.后来的苏格兰警察僵尸都蛮有意思的某些人运气非常悲鄙视下2步就被逮到真是要笑死大家了.阿拉边吃边玩红茶喝喝时间瞬间就晚上了瞧着人都走光了肚子也饿了就在附近找

In [33]:
# 🔧 修正資料設定並執行完整處理流程
print("🔧 修正資料設定...")

# 檢查並設定DataFrame
if 'selected_df' in globals() and selected_df is not None:
    print(f"✅ 找到 selected_df: {len(selected_df)} 筆")
    
    # 轉換為DataFrame
    selected_large_file_df = pd.DataFrame(selected_df)
    
    print(f"📊 DataFrame 建立成功: {selected_large_file_df.shape}")
    print(f"📋 欄位: {list(selected_large_file_df.columns)}")
    
    # 顯示前幾筆範例
    print(f"\n📝 前3筆資料:")
    for i in range(min(3, len(selected_large_file_df))):
        row = selected_large_file_df.iloc[i]
        text = row['text']
        preview = text[:80] + "..." if len(text) > 80 else text
        print(f"第{i+1}筆: {preview}")
    
    print(f"\n🔄 開始執行完整處理流程...")
    
    # Step 1: 句子級別切分
    print(f"\n🔪 步驟1: 執行句子級別切分...")
    
    selected_split_df = process_text_splitting(
        df=selected_large_file_df,
        text_column='text',
        min_length=30,
        max_length=250
    )
    
    if not selected_split_df.empty:
        print(f"\n📊 切分結果:")
        print(f"  原始文本: {len(selected_large_file_df)} 筆")
        print(f"  切分後句子片段: {len(selected_split_df)} 筆")
        print(f"  平均每文本產生: {len(selected_split_df)/len(selected_large_file_df):.1f} 個片段")
        
        # 顯示切分範例
        print(f"\n📝 切分範例 (第1個原始文本):")
        first_orig_idx = selected_split_df['original_index'].iloc[0]
        fragments = selected_split_df[selected_split_df['original_index'] == first_orig_idx]
        original_text = selected_large_file_df.iloc[first_orig_idx]['text']
        
        print(f"原始文本: {original_text}")
        print(f"切分為 {len(fragments)} 個片段:")
        
        for i, (_, row) in enumerate(fragments.iterrows()):
            print(f"  片段{i+1} ({row['fragment_length']}字): {row['text']}")
        print("-" * 80)
        
        # Step 2: 大陸用語評分 (取較小的樣本數進行演示)
        print(f"\n🎯 步驟2: 執行大陸用語評分...")
        
        # 設定評分參數 - 減少樣本數以加快處理
        SAMPLE_SIZE = min(200, len(selected_split_df))  # 取30個片段進行評分演示
        THRESHOLD = 3  # 評分閾值
        
        print(f"📊 評分參數:")
        print(f"  評分樣本數: {SAMPLE_SIZE}")
        print(f"  篩選閾值: {THRESHOLD}/5")
        
        # 執行評分
        results, authentic_results, generic_results = await process_dataset_async_batched(
            df=available_data,
            model_endpoint=MODEL_API_ENDPOINT,
            api_key=OPENWEBUI_API_KEY,
            model_name=MODEL_NAME,
            text_col=text_column,
            sample_size=SAMPLE_SIZE,
            threshold=THRESHOLD,
            batch_size=BATCH_SIZE
        )
        
        # 統計結果
        print(f"\n📈 評分結果統計:")
        print(f"  總評分數: {len(results)}")
        print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
        print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
        if len(results) > 0:
            print(f"  📊 大陸用語比例: {len(authentic_results)/len(results)*100:.1f}%")
        
        # 顯示高質量範例
        if authentic_results:
            print(f"\n🏆 高質量大陸用語片段範例:")
            for i, r in enumerate(authentic_results[:3]):
                print(f"第{i+1}名 (得分:{r['scores']['總分']}/5): {r['text']}")
                if r['features']['mainland_terms']:
                    print(f"  大陸特有詞彙: {', '.join(r['features']['mainland_terms'])}")
                print("-" * 60)
        
        # 儲存結果
        print(f"\n💾 儲存處理結果...")
        full_df, auth_df = save_results(results, authentic_results, generic_results)
        
        # 設定全域變數
        globals()['selected_large_file_df'] = selected_large_file_df
        globals()['selected_split_df'] = selected_split_df
        globals()['selected_mainland_results'] = results
        globals()['selected_authentic_data'] = auth_df
        
        print(f"\n🎉 250筆資料完整處理流程完成！")
        print(f"📊 最終統計:")
        print(f"  原始挑選資料: {len(selected_large_file_df)} 筆")
        print(f"  句子切分後: {len(selected_split_df)} 個片段")
        print(f"  大陸用語評分: {len(results)} 個片段")
        print(f"  高質量大陸用語: {len(authentic_results)} 個片段")
        
        print(f"\n📋 可用變數:")
        print(f"  selected_large_file_df: 原始挑選的250筆資料")
        print(f"  selected_split_df: 句子切分後的片段資料")
        print(f"  selected_mainland_results: 大陸用語評分結果")
        if auth_df is not None:
            print(f"  selected_authentic_data: 高質量大陸用語片段 ({len(auth_df)} 筆)")
        
    else:
        print("❌ 句子切分失敗，無法進行後續處理")

else:
    print("❌ 沒有找到 selected_df")
    print("💡 請先執行前面的資料挑選步驟")

print("="*80)

🔧 修正資料設定...
✅ 找到 selected_df: 250 筆
📊 DataFrame 建立成功: (250, 2)
📋 欄位: ['text', 'text_length']

📝 前3筆資料:
第1筆: 一元秒杀活动：10月29日12：00图书商城准时开启！活动介绍 为了感谢长久以来一直关注并支持我们的粉丝，机械工业出版社电工电子分社兹定于10月15日、10月2...
第2筆: 孩子入门用的仔细看了有点难度大人读起来还可以孩子是读不懂的
第3筆: 午评：沪指收复4000点 两市个股普涨今日早盘，沪深股指小幅低开后冲高，沪指收复4000点大关。截至午间收盘，沪指报4012.97点，涨幅1.40%；深成指报1...

🔄 開始執行完整處理流程...

🔪 步驟1: 執行句子級別切分...
📊 開始處理文本切分...
  原始資料筆數: 250
  文本欄位: text
  句子片段長度範圍: 30-250 字


切分進度: 100%|██████████| 250/250 [00:00<00:00, 435.64it/s]



✅ 文本切分完成！
📈 切分統計:
  處理文本數: 193
  生成句子片段數: 892
  平均每文本片段數: 4.6

📏 片段長度統計:
  平均長度: 63.2 字
  最短片段: 30 字
  最長片段: 413 字
  中位數長度: 51.0 字

📊 片段長度分布:
  短片段 (10-20字): 0 個 (0.0%)
  中短片段 (20-30字): 0 個 (0.0%)
  中等片段 (30-40字): 240 個 (26.9%)
  長片段 (40-50字): 186 個 (20.9%)
  超長片段 (50-100字): 357 個 (40.0%)

📊 切分結果:
  原始文本: 250 筆
  切分後句子片段: 892 筆
  平均每文本產生: 3.6 個片段

📝 切分範例 (第1個原始文本):
原始文本: 一元秒杀活动：10月29日12：00图书商城准时开启！活动介绍 为了感谢长久以来一直关注并支持我们的粉丝，机械工业出版社电工电子分社兹定于10月15日、10月22日、10月29日，在“机械工业出版社E视界”官方微书城开展三期1元秒杀活动，每期提供5种畅销品种共20本图书。还等什么，准备好零钱，等着明天中午12：00的钟声吧！活动书单 本期书单（10月29日12:00） 活动详解 1.点击 “E社群”→“图书商城”进入微书城。2.点击相应专题图片进入秒杀活动。温馨提示：请大家待活动开启后再进入专题进行秒杀！每人只能以秒杀价抢购1本图书！
切分為 2 個片段:
  片段1 (109字): 活动介绍 为了感谢长久以来一直关注并支持我们的粉丝，机械工业出版社电工电子分社兹定于10月15日、10月22日、10月29日，在“机械工业出版社E视界”官方微书城开展三期1元秒杀活动，每期提供5种畅销品种共20本图书。
  片段2 (51字): 活动书单 本期书单（10月29日12:00） 活动详解 1.点击 “E社群”→“图书商城”进入微书城。
--------------------------------------------------------------------------------

🎯 步驟2: 執行大陸用語評分...
📊 評分參數:
  評分樣本數: 200
  篩選閾值: 3/5
📊 處理資料集：3870 筆


非同步批次推論: 100%|██████████| 10/10 [00:12<00:00,  1.24s/it]


📈 評分結果統計:
  總評分數: 200
  ✅ 真正大陸用語: 31 筆
  🗑️ 通用簡體中文: 169 筆
  📊 大陸用語比例: 15.5%

🏆 高質量大陸用語片段範例:
第1名 (得分:3/5): 现在已经成了吃小海鲜就会想到的地方因为去旁边的锅一烧而路过的地方，本以为是倪式海泰旗下的会很贵，谁知道一进去就被菜单吓了一跳，还真是便宜的说现在已经吃遍了这里所有的菜和主食，写下自己不喜欢的，剩下的都OK：卤水蚍蛴香螺因为我比较喜欢吃酱爆口味的。
------------------------------------------------------------
第2名 (得分:3/5): 服务员的态度我觉得也挺好的，人多，上菜慢，很多桌都有催的，但她们不是在顾客面前应付一下，转身干自己的事，而是真正的帮每桌去催，然后回来告知几分钟会上。
  大陸特有詞彙: 挺
------------------------------------------------------------
第3名 (得分:3/5): 4 开始学生肯定不多，慢慢来，口碑好了，不用宣传都有人找上门来。
------------------------------------------------------------

💾 儲存處理結果...
💾 儲存完成:
  📄 完整結果: mainland_filtering_complete_20250908_205725.csv
  ✅ 高質量句子片段數據: authentic_mainland_texts_20250908_205725.csv
  📋 JSON格式: authentic_mainland_texts_20250908_205725.json

🎉 250筆資料完整處理流程完成！
📊 最終統計:
  原始挑選資料: 250 筆
  句子切分後: 892 個片段
  大陸用語評分: 200 個片段
  高質量大陸用語: 31 個片段

📋 可用變數:
  selected_large_file_df: 原始挑選的250筆資料
  selected_split_df: 句子切分後的片段資料
  selected_mainland_results: 大陸用語評分結果
  selected_authentic_data: 

In [34]:
# 🔧 修正資料設定並執行完整處理流程
print("🔧 修正資料設定...")

# 檢查並設定DataFrame
if 'selected_large_file_df' in globals() and selected_large_file_df is not None:
    print(f"✅ 找到 selected_large_file_df: {len(selected_large_file_df)} 筆")
    
    # 顯示前幾筆範例
    print(f"\n📝 前3筆資料:")
    for i in range(min(3, len(selected_large_file_df))):
        row = selected_large_file_df.iloc[i]
        text = row['text']
        preview = text[:80] + "..." if len(text) > 80 else text
        print(f"第{i+1}筆: {preview}")
    
    print(f"\n🔄 開始執行完整處理流程...")
    
    # Step 1: 句子級別切分
    print(f"\n🔪 步驟1: 執行句子級別切分...")
    
    selected_split_df = process_text_splitting(
        df=selected_large_file_df,
        text_column='text',
        min_length=30,
        max_length=250
    )
    
    if not selected_split_df.empty:
        print(f"\n📊 切分結果:")
        print(f"  原始文本: {len(selected_large_file_df)} 筆")
        print(f"  切分後句子片段: {len(selected_split_df)} 筆")
        print(f"  平均每文本產生: {len(selected_split_df)/len(selected_large_file_df):.1f} 個片段")
        
        # 顯示切分範例
        print(f"\n📝 切分範例 (第1個原始文本):")
        first_orig_idx = selected_split_df['original_index'].iloc[0]
        fragments = selected_split_df[selected_split_df['original_index'] == first_orig_idx]
        original_text = selected_large_file_df.iloc[first_orig_idx]['text']
        
        print(f"原始文本: {original_text}")
        print(f"切分為 {len(fragments)} 個片段:")
        
        for i, (_, row) in enumerate(fragments.iterrows()):
            print(f"  片段{i+1} ({row['fragment_length']}字): {row['text']}")
        print("-" * 80)
        
        # Step 2: 大陸用語評分 (取較小的樣本數進行演示)
        print(f"\n🎯 步驟2: 執行大陸用語評分...")
        
        # 設定評分參數 - 減少樣本數以加快處理
        SAMPLE_SIZE = min(200, len(selected_split_df))  # 取30個片段進行評分演示
        THRESHOLD = 3  # 評分閾值
        
        print(f"📊 評分參數:")
        print(f"  評分樣本數: {SAMPLE_SIZE}")
        print(f"  篩選閾值: {THRESHOLD}/5")
        
        # 執行評分
        results, authentic_results, generic_results = await process_dataset_async_batched(
            df=selected_large_file_df,  # 使用 selected_large_file_df 資料
            model_endpoint=MODEL_API_ENDPOINT,
            api_key=OPENWEBUI_API_KEY,
            model_name=MODEL_NAME,
            text_col='text',
            sample_size=SAMPLE_SIZE,
            threshold=THRESHOLD,
            batch_size=BATCH_SIZE
        )
        
        # 統計結果
        print(f"\n📈 評分結果統計:")
        print(f"  總評分數: {len(results)}")
        print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
        print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
        if len(results) > 0:
            print(f"  📊 大陸用語比例: {len(authentic_results)/len(results)*100:.1f}%")
        
        # 顯示高質量範例
        if authentic_results:
            print(f"\n🏆 高質量大陸用語片段範例:")
            for i, r in enumerate(authentic_results[:3]):
                print(f"第{i+1}名 (得分:{r['scores']['總分']}/5): {r['text']}")
                if r['features']['mainland_terms']:
                    print(f"  大陸特有詞彙: {', '.join(r['features']['mainland_terms'])}")
                print("-" * 60)
        
        # 儲存結果
        print(f"\n💾 儲存處理結果...")
        full_df, auth_df = save_results(results, authentic_results, generic_results)
        
        # 設定全域變數
        globals()['selected_split_df'] = selected_split_df
        globals()['selected_mainland_results'] = results
        globals()['selected_authentic_data'] = auth_df
        
        print(f"\n🎉 250筆資料完整處理流程完成！")
        print(f"📊 最終統計:")
        print(f"  原始挑選資料: {len(selected_large_file_df)} 筆")
        print(f"  句子切分後: {len(selected_split_df)} 個片段")
        print(f"  大陸用語評分: {len(results)} 個片段")
        print(f"  高質量大陸用語: {len(authentic_results)} 個片段")
        
        print(f"\n📋 可用變數:")
        print(f"  selected_large_file_df: 原始挑選的250筆資料")
        print(f"  selected_split_df: 句子切分後的片段資料")
        print(f"  selected_mainland_results: 大陸用語評分結果")
        if auth_df is not None:
            print(f"  selected_authentic_data: 高質量大陸用語片段 ({len(auth_df)} 筆)")
        
    else:
        print("❌ 句子切分失敗，無法進行後續處理")

else:
    print("❌ 沒有找到 selected_large_file_df")
    print("💡 請先執行前面的資料挑選步驟")

print("="*80)


🔧 修正資料設定...
✅ 找到 selected_large_file_df: 250 筆

📝 前3筆資料:
第1筆: 一元秒杀活动：10月29日12：00图书商城准时开启！活动介绍 为了感谢长久以来一直关注并支持我们的粉丝，机械工业出版社电工电子分社兹定于10月15日、10月2...
第2筆: 孩子入门用的仔细看了有点难度大人读起来还可以孩子是读不懂的
第3筆: 午评：沪指收复4000点 两市个股普涨今日早盘，沪深股指小幅低开后冲高，沪指收复4000点大关。截至午间收盘，沪指报4012.97点，涨幅1.40%；深成指报1...

🔄 開始執行完整處理流程...

🔪 步驟1: 執行句子級別切分...
📊 開始處理文本切分...
  原始資料筆數: 250
  文本欄位: text
  句子片段長度範圍: 30-250 字


切分進度: 100%|██████████| 250/250 [00:00<00:00, 417.26it/s]



✅ 文本切分完成！
📈 切分統計:
  處理文本數: 193
  生成句子片段數: 892
  平均每文本片段數: 4.6

📏 片段長度統計:
  平均長度: 63.2 字
  最短片段: 30 字
  最長片段: 413 字
  中位數長度: 51.0 字

📊 片段長度分布:
  短片段 (10-20字): 0 個 (0.0%)
  中短片段 (20-30字): 0 個 (0.0%)
  中等片段 (30-40字): 240 個 (26.9%)
  長片段 (40-50字): 186 個 (20.9%)
  超長片段 (50-100字): 357 個 (40.0%)

📊 切分結果:
  原始文本: 250 筆
  切分後句子片段: 892 筆
  平均每文本產生: 3.6 個片段

📝 切分範例 (第1個原始文本):
原始文本: 一元秒杀活动：10月29日12：00图书商城准时开启！活动介绍 为了感谢长久以来一直关注并支持我们的粉丝，机械工业出版社电工电子分社兹定于10月15日、10月22日、10月29日，在“机械工业出版社E视界”官方微书城开展三期1元秒杀活动，每期提供5种畅销品种共20本图书。还等什么，准备好零钱，等着明天中午12：00的钟声吧！活动书单 本期书单（10月29日12:00） 活动详解 1.点击 “E社群”→“图书商城”进入微书城。2.点击相应专题图片进入秒杀活动。温馨提示：请大家待活动开启后再进入专题进行秒杀！每人只能以秒杀价抢购1本图书！
切分為 2 個片段:
  片段1 (109字): 活动介绍 为了感谢长久以来一直关注并支持我们的粉丝，机械工业出版社电工电子分社兹定于10月15日、10月22日、10月29日，在“机械工业出版社E视界”官方微书城开展三期1元秒杀活动，每期提供5种畅销品种共20本图书。
  片段2 (51字): 活动书单 本期书单（10月29日12:00） 活动详解 1.点击 “E社群”→“图书商城”进入微书城。
--------------------------------------------------------------------------------

🎯 步驟2: 執行大陸用語評分...
📊 評分參數:
  評分樣本數: 200
  篩選閾值: 3/5
📊 處理資料集：250 筆


非同步批次推論: 100%|██████████| 10/10 [00:32<00:00,  3.23s/it]


📈 評分結果統計:
  總評分數: 200
  ✅ 真正大陸用語: 59 筆
  🗑️ 通用簡體中文: 141 筆
  📊 大陸用語比例: 29.5%

🏆 高質量大陸用語片段範例:
第1名 (得分:3/5): 卓越算你狠以下评价与本书无关。全部送给由卓越网销售机械工业出版社出版的、潘金贵翻译的《算法导论》全部送给无耻的卓越竟然屏蔽差评俺没得办法只好借所有俺买过的书来发言了原著确实当得上一个好字电子版的原著看了一遍了准备弄本纸质书精度几遍。原意买英文版的。收到书之后发现买错了。只能将就着看。过年放假在家里狠啃了一段时间。非常非常没有想到这竟然是一本让人无法将就的书有很多人评论好很好。这证明他她压根就没看浏览前几页之后就来唱赞歌了。首先说说翻译质量和大多数的所谓译著一样感觉是我自己对照字典弄出来的东西。当然你自己也能对照字典弄出来。学者们研究任务繁重这个俺就不多说了。谁让俺自个稀里糊涂买了个译本。更严重的问题 一、94页之后竟然是111页 二、缺第八章全部 三、缺第九章前两节 四、第十章前三节重复出现两次 五、第九章的练习题重复出现在第十章。后面不知道还有多少问题不想往下看了。卓越你给我退货吧。从此之后再也不买机械工业出版社的东西了。没想到机械工业出版社能把一本好书糟蹋成这样一群XXXX卓越你得给我退货。删俺的评价不给俺电话号码和信箱 现在俺不要退货了俺要跟你死磕
------------------------------------------------------------
第2名 (得分:3/5): 现在穆黑是主流声音吗？绿教大概什么时间会引起世俗社会的愤怒？求求您可别吓人了，穆黑？据统计全世界有17亿穆斯林，一共70亿人，4个人就有一个穆斯林，有比穆斯林人数上更占主流的群体？穆黑算个屁，穆黑有1亿不？小心说话，分分钟弄死你，再敢政治不正确爆破你全家
------------------------------------------------------------
第3名 (得分:3/5): 不错挺结实的不错挺结实的就是组装有点儿费劲
  大陸特有詞彙: 挺
------------------------------------------------------------

💾 儲存處理結果...
💾 儲存完成:
  📄 完整結果: main

In [36]:
# 🎯 簡化版結果總結和儲存
print("🎯 整理和儲存最終結果...")

# 確保有評分結果
if 'results' in locals() and 'authentic_results' in locals():
    print(f"✅ 找到評分結果")
    
    # 創建完整結果DataFrame
    full_results_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_results_data.append(row)
    
    full_results_df = pd.DataFrame(full_results_data)
    
    # 創建高質量大陸用語DataFrame
    if authentic_results:
        authentic_data = []
        for r in authentic_results:
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length'],
                'category': r['category']
            }
            
            # 添加詳細評分
            if r['scores']:
                for key, value in r['scores'].items():
                    if key != '總分':
                        auth_row[f'score_{key}'] = value
            
            authentic_data.append(auth_row)
        
        authentic_df = pd.DataFrame(authentic_data)
    else:
        authentic_df = pd.DataFrame()
    
    # 儲存結果
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = "evaluation_results"
    os.makedirs(results_dir, exist_ok=True)
    
    # 儲存完整結果
    full_results_file = f"{results_dir}/full_evaluation_results_{timestamp}.csv"
    full_results_df.to_csv(full_results_file, index=False, encoding='utf-8-sig')
    
    # 儲存高質量結果
    if not authentic_df.empty:
        authentic_file = f"{results_dir}/high_quality_mainland_texts_{timestamp}.csv"
        authentic_df.to_csv(authentic_file, index=False, encoding='utf-8-sig')
        
        authentic_json = f"{results_dir}/high_quality_mainland_texts_{timestamp}.json"
        authentic_df.to_json(authentic_json, orient='records', force_ascii=False, indent=2)
    
    # 創建處理摘要
    summary = {
        "processing_timestamp": timestamp,
        "source_file": "/Users/edwardhuang/Documents/GitHub/LLM_Aug/data/CLUECorpusSmall.txt",
        "original_samples": len(selected_large_file_df),
        "split_fragments": len(selected_split_df),
        "evaluated_fragments": len(results),
        "high_quality_mainland": len(authentic_results),
        "generic_chinese": len(generic_results),
        "mainland_percentage": len(authentic_results)/len(results)*100 if len(results) > 0 else 0,
        "fragment_length_stats": {
            "mean": float(selected_split_df['fragment_length'].mean()),
            "min": int(selected_split_df['fragment_length'].min()),
            "max": int(selected_split_df['fragment_length'].max()),
            "median": float(selected_split_df['fragment_length'].median())
        },
        "evaluation_sample_size": len(results),
        "threshold_used": 3
    }
    
    # 儲存摘要
    summary_file = f"{results_dir}/processing_summary_{timestamp}.json"
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 結果已成功儲存:")
    print(f"  📊 完整評分結果: {full_results_file}")
    if not authentic_df.empty:
        print(f"  🏆 高質量大陸用語: {authentic_file}")
        print(f"  📋 JSON格式: {authentic_json}")
    print(f"  📋 處理摘要: {summary_file}")
    
    # 顯示檔案大小
    print(f"\n📁 檔案大小:")
    for filename in [full_results_file, authentic_file if not authentic_df.empty else None, summary_file]:
        if filename and os.path.exists(filename):
            size_kb = os.path.getsize(filename) / 1024
            print(f"  {os.path.basename(filename)}: {size_kb:.2f} KB")
    
    # 設定全域變數
    globals()['selected_large_file_df'] = selected_large_file_df
    globals()['selected_split_df'] = selected_split_df
    globals()['selected_mainland_results'] = results
    globals()['selected_authentic_data'] = authentic_df
    globals()['evaluation_summary'] = summary
    
    print(f"\n🎉 250筆資料評分處理完成！")
    print(f"\n📊 最終統計摘要:")
    print(f"  📂 原始大文件: CLUECorpusSmall.txt")
    print(f"  🎲 隨機挑選: {summary['original_samples']} 筆資料")
    print(f"  🔪 句子切分: {summary['split_fragments']} 個片段")
    print(f"  🎯 評分處理: {summary['evaluated_fragments']} 個片段")
    print(f"  ✅ 高質量大陸用語: {summary['high_quality_mainland']} 個片段")
    print(f"  📈 大陸用語比例: {summary['mainland_percentage']:.1f}%")
    
    if authentic_results:
        print(f"\n🏆 發現的高質量大陸用語片段:")
        for i, r in enumerate(authentic_results):
            print(f"  {i+1}. (得分 {r['scores']['總分']}/5): {r['text']}")
            if r['features']['mainland_terms']:
                print(f"     大陸特有詞彙: {', '.join(r['features']['mainland_terms'])}")
    
    print(f"\n📋 可用變數:")
    print(f"  selected_large_file_df: 原始250筆資料")
    print(f"  selected_split_df: 切分後的句子片段")
    print(f"  selected_mainland_results: 評分結果")
    print(f"  selected_authentic_data: 高質量大陸用語")
    print(f"  evaluation_summary: 處理摘要")

else:
    print("❌ 沒有找到評分結果")

print("="*80)

🎯 整理和儲存最終結果...
✅ 找到評分結果

💾 結果已成功儲存:
  📊 完整評分結果: evaluation_results/full_evaluation_results_20250908_205810.csv
  🏆 高質量大陸用語: evaluation_results/high_quality_mainland_texts_20250908_205810.csv
  📋 JSON格式: evaluation_results/high_quality_mainland_texts_20250908_205810.json
  📋 處理摘要: evaluation_results/processing_summary_20250908_205810.json

📁 檔案大小:
  full_evaluation_results_20250908_205810.csv: 161.77 KB
  high_quality_mainland_texts_20250908_205810.csv: 42.22 KB
  processing_summary_20250908_205810.json: 0.47 KB

🎉 250筆資料評分處理完成！

📊 最終統計摘要:
  📂 原始大文件: CLUECorpusSmall.txt
  🎲 隨機挑選: 250 筆資料
  🔪 句子切分: 892 個片段
  🎯 評分處理: 200 個片段
  ✅ 高質量大陸用語: 59 個片段
  📈 大陸用語比例: 29.5%

🏆 發現的高質量大陸用語片段:
  1. (得分 3/5): 卓越算你狠以下评价与本书无关。全部送给由卓越网销售机械工业出版社出版的、潘金贵翻译的《算法导论》全部送给无耻的卓越竟然屏蔽差评俺没得办法只好借所有俺买过的书来发言了原著确实当得上一个好字电子版的原著看了一遍了准备弄本纸质书精度几遍。原意买英文版的。收到书之后发现买错了。只能将就着看。过年放假在家里狠啃了一段时间。非常非常没有想到这竟然是一本让人无法将就的书有很多人评论好很好。这证明他她压根就没看浏览前几页之后就来唱赞歌了。首先说说翻译质量和大多数的所谓译著一样感觉是我自己对照字典弄出来的东西。当然你自己也能对照字典弄出来。学者们研究任务繁重这个俺就不多说了。谁让俺自个稀里糊涂买了个译本。更严重